In [1]:
from importlib import reload
#reload(Utilities)
#reload(clm)
# NOTE: To reload a class imported as, e.g., 
# from module import class
# One must call:
#   1. import module
#   2. reload module
#   3. from module import class

import sys, os
import re

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype, is_datetime64_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns
from packaging import version

import itertools
import copy
import pyodbc
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
import CommonLearningMethods as clm
#-----
from MeterPremise import MeterPremise
#-----
from AMI_SQL import AMI_SQL
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
from AMIUsgInst_SQL import AMIUsgInst_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from AMIEndEvents import AMIEndEvents
from AMIUsgInst import AMIUsgInst
from DOVSOutages import DOVSOutages
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
import TableInfos
from TableInfos import TableInfo
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
#sys.path.insert(0, os.path.join(os.path.realpath('..'), 'Utilities'))
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
from Utilities_df import DFConstructType
import Utilities_dt
import Plot_Box_sns
import GrubbsTest

In [2]:
def set_random_date_interval_for_each_entry_in_df(
    df, 
    date_0, 
    date_1, 
    window_width, 
    rand_seed=None, 
    placement_cols=['start_date', 'end_date'], 
    inplace=True
):
    assert(len(placement_cols)==2)
    if not inplace:
        df = df.copy()
    df[placement_cols] = df.apply(
        lambda _: Utilities_dt.get_random_datetime_interval_between(
            date_0=date_0, 
            date_1=date_1, 
            window_width=window_width, 
            rand_seed=rand_seed), 
        axis=1, 
        result_type='expand'
    )
    return df

def set_random_date_interval_in_df(
    df, 
    date_0, 
    date_1, 
    window_width, 
    groupby=None, 
    rand_seed=None, 
    placement_cols=['start_date', 'end_date'], 
    inplace=True
):
    r"""
    Sets single interval for entire df UNLESS groupby is not None
    If groupby is not None, then a single interval is set for EACH GROUP
      - NOTE: In this case, rand_seed must be an int (or other object to which 
      adding an int makes sense)
    
    It is not suggested, but if one wanted a unique interval for each entry in df,
    one could use this function after grouping df by unique indentifier (row index, etc)
      - Or, one could use set_random_date_interval_for_each_entry_in_df
    """
    #-------------------------
    assert(len(placement_cols)==2)
    if not inplace:
        df = df.copy()
    #-------------------------
    if groupby is not None:
        # Below, groupby should be set to None in set_random_date_interval_in_df, 
        #   as the df.groupby(groupby) essentially sends each group's DataFrame into
        #   this function (and setting groupby=None ensures the code doesn't end up right back here!)
        # Note: inplace already handled above, so inplace=True below
        # Note: A single rand_seed value would make each group have the exact same interval!
        #       Therefore, if rand_seed is set, somehow each call to this function must have a unique
        #       rand_seed value!.  The simplest way to achieve this is to use rand_seed = rand_seed+x.shape[0]
        #       where x is the df for a particular group.  Note that groups with the same number of entries will
        #       have the same seed, and therefore the same time iterval.
        #       This functionality is intended for debugging/development, therefore rand_seed should be set
        #       to done when run for production.
        if rand_seed is not None:
            try:
                rand_seed+1
            except:
                assert(0)
        df=df.groupby(groupby).apply(
            lambda x: set_random_date_interval_in_df(
                df=x, 
                date_0=date_0, 
                date_1=date_1, 
                window_width=window_width, 
                groupby=None, 
                rand_seed=rand_seed if rand_seed is None else x.shape[0]+rand_seed, 
                placement_cols=placement_cols,
                inplace=True
            )
        )
        return df
    #-------------------------
    interval = Utilities_dt.get_random_datetime_interval_between(
        date_0=date_0, 
        date_1=date_1, 
        window_width=window_width, 
        rand_seed=rand_seed
    )
    df[placement_cols] = interval
    return df

In [3]:
from ast import literal_eval
def read_consolidated_df_from_csv(
    file_path, 
    list_cols
):
    r"""
    When stored as a CSV, any list columns are retrieved as a strings, instead of as a lists.
    Using ast.literal_eval fixes that
    """
    df = pd.read_csv(file_path, dtype=str)
    for col in list_cols:
        df[col] = df[col].apply(lambda x: literal_eval(x))
    return df

# -----------------------------------------------------------------------------------------------
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# -----------------------------------------------------------------------------------------------

In [4]:
# save_dir_base_csvs contains path to save e.g., df_mp_all, df_mp_no_outg, etc.
save_dfs_to_csv = False
read_dfs_from_csv = True
save_dir_base_csvs = os.path.join(Utilities.get_local_data_dir(), r'dovs_and_end_events_data')

assert(save_dfs_to_csv+read_dfs_from_csv <=1) # Should never both read and write!

#-------------------------
# save_args contains path information for final end_events files
save_args = dict(save_to_file=True, 
                 save_dir = r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\EndEvents_NoOutg', 
                 save_name=r'end_events.csv', 
                 index=True)
batch_size=10
verbose=True
n_update=1

In [5]:
fig_num = 0

In [6]:
if not read_dfs_from_csv:
    conn_outages = Utilities.get_utldb01p_oracle_connection()
    conn_aws = Utilities.get_athena_prod_aws_connection()

In [7]:
# date_0 = '2020-01-01'
# date_1 = '2020-12-31'

date_0 = '2022-01-01'
date_1 = '2022-05-31'

window_width = datetime.timedelta(days=30)

run_using_slim = True

# In general, the full and slim dfs will have different search time
# They can be made equal by setting this to True
# Although, it takes a while to run, and I don't suggest it for large datasets
make_search_times_equal_between_full_and_slim = False

In [8]:
save_subdir_csvs = f"{date_0.replace('-','')}_{date_1.replace('-','')}"
save_dir_csvs = os.path.join(save_dir_base_csvs, save_subdir_csvs)
if not os.path.exists(save_dir_csvs):
    os.makedirs(save_dir_csvs)

# -----------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------

# Build no outage collection and df_mp_all
where df_mp_all is a DataFrame containing all meter premise data (from OH)

In [9]:
cols_of_interest_met_prem = TableInfos.MeterPremise_TI.std_columns_of_interest
cols_of_interest_met_prem.append('curr_acct_cls_cd')

cols_of_interest_end_dev_event = TableInfos.AMIEndEvents_TI.std_columns_of_interest

In [10]:
if not read_dfs_from_csv:
    #----------------------------------------------------------------------------------------------------
    # Find transformers which haven't experienced an outage since date_0
    #----------------------------------------------------------------------------------------------------

    #--------------------------------------------------
    # First, find all transformers which HAVE experienced an outage
    sql_outage_location_ids = DOVSOutages_SQL.build_sql_find_outage_xfmrs(
        mjr_mnr_cause=None, 
        include_premise=False, 
        date_range=[date_0, date_1], 
        states='OH'
    )
    sql_stmnt_outage_location_ids = sql_outage_location_ids.get_sql_statement()
    print(f'sql_stmnt_outage_location_ids:\n{sql_stmnt_outage_location_ids}\n\n')
    df_outage_location_ids_OG = pd.read_sql_query(sql_stmnt_outage_location_ids, conn_outages)

    #--------------------------------------------------
    # Now, find those which HAVEN'T experienced an outage

    # When df_outage_location_ids_OG is large
    #   i.e., when the date range used to find outages is large
    #   It is actually faster to grab ALL meters from default.meter_premise and then filter using pandas
    #     as opposed to using NOT IN and filtering with SQL

    sql_mp_all = MeterPremise.build_sql_meter_premise(cols_of_interest_met_prem, states=['OH']).get_sql_statement()
    print(f'sql_mp_all:\n{sql_mp_all}\n\n')
    df_mp_all_OG = pd.read_sql(sql_mp_all, conn_aws) 

    #--------------------------------------------------
    # Get no outage collection as those in df_mp_all_OG without trsf_pole_nb matching those in df_outage_location_ids_OG
    df_mp_no_outg_OG = df_mp_all_OG[~df_mp_all_OG['trsf_pole_nb'].isin(df_outage_location_ids_OG['LOCATION_ID'].tolist())]

    #--------------------------------------------------
    # Get outage collection as those in df_mp_all_OG with trsf_pole_nb matching those in df_outage_location_ids_OG
    df_mp_outg_OG = df_mp_all_OG[df_mp_all_OG['trsf_pole_nb'].isin(df_outage_location_ids_OG['LOCATION_ID'].tolist())]


    #----------------------------------------------------------------------------------------------------
    # Find PNs with at least one end event in the time frame, and weed out those without from df_mp_no_outg_OG
    #----------------------------------------------------------------------------------------------------
    # Looking for PNs which have no entries takes a great deal of time.  This chops down the time to create the CSVs dramatically 
    #   by reducing the number of empty (or nearly empty) files
    # NOTE: Should use PNs instead of SNs

    # NOTE FROM WHEN I WAS PREVIOUSLY USING SNS:
    # WHY is the length of df_mp_no_outg_OG not equal to that of sns_with_end_events?
    #   Main reason is because not all meters with some sort of event suffer an outage!!!!!
    #   But, also because df_mp_no_outg_OG only includes meters which are currently active, whereas sns_with_end_events
    #     can have meters which were active at the time of the event but are no longer active
    pns_with_end_events = AMIEndEvents.get_end_event_distinct_fields(
        date_0=date_0, 
        date_1=date_1, 
        fields=['aep_premise_nb'], 
        are_datetime=False, 
        addtnl_build_sql_function_kwargs = dict(opco='oh')
    )

    print(f'BEFORE: df_mp_no_outg_OG.shape = {df_mp_no_outg_OG.shape}')
    start=time.time()
    df_mp_no_outg_OG = df_mp_no_outg_OG[df_mp_no_outg_OG['prem_nb'].isin(pns_with_end_events['aep_premise_nb'].tolist())]
    print(time.time()-start)
    print(f'AFTER: df_mp_no_outg_OG.shape = {df_mp_no_outg_OG.shape}')

    #----------------------------------------------------------------------------------------------------
    # Build slim version of df_mp_no_outg_OG
    #----------------------------------------------------------------------------------------------------
    cols_shared_by_group = [
        'state_cd', 'circuit_nb', 'circuit_nm', 'station_nb',
        'station_nm', 'xfmr_nb'
    ]
    cols_to_collect_in_lists = ['mfr_devc_ser_nbr', 'prem_nb']
    rename_cols = {'mfr_devc_ser_nbr':'serial_numbers', 'prem_nb':'premise_nbs'}
    # rename_cols = {'mfr_devc_ser_nbr':'SNs', 'prem_nb':'PNs'}

    print('\n\nBuilding slim version of df_mp_no_outg_OG')
    start=time.time()
    df_mp_no_outg_slim_OG = Utilities_df.consolidate_df(
        df=df_mp_no_outg_OG, 
        groupby_col='trsf_pole_nb', 
        cols_shared_by_group=cols_shared_by_group, 
        cols_to_collect_in_lists=cols_to_collect_in_lists, 
        include_groupby_col_in_output_cols=True, 
        allow_duplicates_in_lists=False, 
        recover_uniqueness_violators=True, 
        rename_cols=rename_cols, 
        verbose=True
    )
    print(time.time()-start) 
    
    #----------------------------------------------------------------------------------------------------
    # Set random times to search for events within date_0 and date_1, and having a period width of window_width
    #----------------------------------------------------------------------------------------------------
    print('Setting times in slim')
    start = time.time()
    df_mp_no_outg_slim_OG = set_random_date_interval_for_each_entry_in_df(
        df=df_mp_no_outg_slim_OG, 
        date_0=date_0, 
        date_1=date_1, 
        window_width=window_width, 
        rand_seed=None, 
        placement_cols=['start_date', 'end_date']
    )
    print(time.time()-start)
    #--------------------------------------------------
    print('Setting times in full')
    start = time.time()
    if make_search_times_equal_between_full_and_slim:
        print('Making search times equal to those in slim')
        df_mp_no_outg_OG['start_date'] = np.nan
        df_mp_no_outg_OG['end_date']   = np.nan
        for idx, df in df_mp_no_outg_OG.groupby('trsf_pole_nb'):
            try:
                start_date = df_mp_no_outg_slim_OG.loc[df['trsf_pole_nb'].unique()[0]]['start_date']
                end_date   = df_mp_no_outg_slim_OG.loc[df['trsf_pole_nb'].unique()[0]]['end_date']
            except:
                print(f'Not found for {idx}')
                start_date, end_date = Utilities_dt.get_random_datetime_interval_between(
                    date_0=date_0, 
                    date_1=date_1, 
                    window_width=window_width, 
                    rand_seed=None
                )
            df_mp_no_outg_OG.loc[idx, 'start_date'] = start_date
            df_mp_no_outg_OG.loc[idx, 'end_date']   = end_date
    else:
        df_mp_no_outg_OG = set_random_date_interval_in_df(
            df=df_mp_no_outg_OG, 
            date_0=date_0, 
            date_1=date_1, 
            window_width=window_width, 
            groupby='trsf_pole_nb', 
            rand_seed=None, 
            placement_cols=['start_date', 'end_date']
        )
    print(time.time()-start)
    #-------------------------
    # If I'm looking at a single year, with a window_width of 30 days, there are only
    # 365-30=335 possible unique start dates, regardless of number of groups.
    # So, there are bound to be repeats!
    print(f"df_mp_no_outg_OG['circuit_nb'].nunique(): {df_mp_no_outg_OG['circuit_nb'].nunique()}")
    print(f"df_mp_no_outg_OG['trsf_pole_nb'].nunique(): {df_mp_no_outg_OG['trsf_pole_nb'].nunique()}")
    print(f"df_mp_no_outg_OG['start_date'].nunique(): {df_mp_no_outg_OG['start_date'].nunique()}")
    print('\n')
    print(f"df_mp_no_outg_slim_OG.index.nunique(): {df_mp_no_outg_slim_OG.index.nunique()}")
    print(f"df_mp_no_outg_slim_OG['start_date'].nunique(): {df_mp_no_outg_slim_OG['start_date'].nunique()}")
    print('\n\n\n')
    print(f"df_mp_no_outg_OG['mfr_devc_ser_nbr'].nunique(): {df_mp_no_outg_OG['mfr_devc_ser_nbr'].nunique()}")
    print(f"df_mp_no_outg_OG['prem_nb'].nunique(): {df_mp_no_outg_OG['prem_nb'].nunique()}")
    print(f"Diff: {df_mp_no_outg_OG['mfr_devc_ser_nbr'].nunique()-df_mp_no_outg_OG['prem_nb'].nunique()}")
    print('\n')
    print(f"df_mp_no_outg_slim_OG['serial_numbers'].apply(len).sum(): {df_mp_no_outg_slim_OG['serial_numbers'].apply(len).sum()}")
    print(f"df_mp_no_outg_slim_OG['premise_nbs'].apply(len).sum(): {df_mp_no_outg_slim_OG['premise_nbs'].apply(len).sum()}")
    print(f"Diff: {df_mp_no_outg_slim_OG['serial_numbers'].apply(len).sum()-df_mp_no_outg_slim_OG['premise_nbs'].apply(len).sum()}")
    #-------------------------
    df_mp_no_outg_OG = df_mp_no_outg_OG.set_index('mfr_devc_ser_nbr', drop=False)
    df_mp_no_outg_OG.index.name = 'idx'
    #-----
    df_mp_no_outg_OG = df_mp_no_outg_OG.sort_values(by=['trsf_pole_nb', 'mfr_devc_ser_nbr'])
    
#----------------------------------------------------------------------------------------------------
#----------------------------------------------------------------------------------------------------
else:
    if not run_using_slim:
        df_outage_location_ids_OG = pd.read_csv(os.path.join(save_dir_csvs, 'df_outage_location_ids_OG.csv'), dtype=str)
        df_mp_all_OG = pd.read_csv(os.path.join(save_dir_csvs, 'df_mp_all_OG.csv'), dtype=str)
        df_mp_no_outg_OG = pd.read_csv(os.path.join(save_dir_csvs, 'df_mp_no_outg_OG.csv'), dtype=str)
        df_mp_outg_OG = pd.read_csv(os.path.join(save_dir_csvs, 'df_mp_outg_OG.csv'), dtype=str)
    # No harm in always loading slim, as it is small compared to others
#     df_mp_no_outg_slim_OG = pd.read_csv(os.path.join(save_dir_csvs, 'df_mp_no_outg_slim_OG.csv'), dtype=str)
    df_mp_no_outg_slim_OG = read_consolidated_df_from_csv(
        file_path=os.path.join(save_dir_csvs, 'df_mp_no_outg_slim_OG.csv'), 
        list_cols = ['serial_numbers', 'premise_nbs']
    )

In [11]:
if save_dfs_to_csv:
    df_outage_location_ids_OG.to_csv(os.path.join(save_dir_csvs, 'df_outage_location_ids_OG.csv'), index=False)
    df_mp_all_OG.to_csv(os.path.join(save_dir_csvs, 'df_mp_all_OG.csv'), index=False)
    df_mp_no_outg_OG.to_csv(os.path.join(save_dir_csvs, 'df_mp_no_outg_OG.csv'), index=False)
    df_mp_outg_OG.to_csv(os.path.join(save_dir_csvs, 'df_mp_outg_OG.csv'), index=False)
    df_mp_no_outg_slim_OG.to_csv(os.path.join(save_dir_csvs, 'df_mp_no_outg_slim_OG.csv'), index=False)

In [12]:
if not read_dfs_from_csv or not run_using_slim:
    print(f'df_outage_location_ids_OG.shape: {df_outage_location_ids_OG.shape}')
    print(f'df_mp_all_OG.shape: {df_mp_all_OG.shape}')
    print(f'df_mp_outg_OG.shape: {df_mp_outg_OG.shape}')
    print(f'df_mp_no_outg_OG.shape: {df_mp_no_outg_OG.shape}')
    #-------------------------
    df_outage_location_ids = df_outage_location_ids_OG.copy()
    #----------
    df_mp_all_OG = clm.remove_prepend_from_columns_in_df(df_mp_all_OG)
    df_mp_all = df_mp_all_OG.copy()
    #----------
    df_mp_no_outg = df_mp_no_outg_OG.copy()
    df_mp_no_outg = Utilities_df.convert_col_types(
        df=df_mp_no_outg, 
        cols_and_types_dict={'annual_kwh':float, 'annual_max_dmnd':float}, 
        to_numeric_errors='coerce', 
        inplace=True
    )
    #----------
    df_mp_outg = df_mp_outg_OG.copy()
#----------
print(f'df_mp_no_outg_slim_OG.shape: {df_mp_no_outg_slim_OG.shape}')
df_mp_no_outg_slim = df_mp_no_outg_slim_OG.copy()

df_mp_no_outg_slim_OG.shape: (192661, 11)


# RUN

In [13]:
conn_aws = Utilities.get_athena_prod_aws_connection()

In [14]:
# if run_using_slim:
#     usg_split_to_CTEs=True
#     df_construct_type=DFConstructType.kRunSqlQuery
#     contstruct_df_args_end_events=None
#     end_events_sql_function_kwargs = dict(
#         cols_of_interest=cols_of_interest_end_dev_event, 
#         df_mp_no_outg=df_mp_no_outg_slim, 
#         split_to_CTEs=usg_split_to_CTEs, 
#         join_mp_args=dict(
#             join_with_CTE=True, 
#             build_mp_kwargs=dict(cols_of_interest=cols_of_interest_met_prem), 
#             join_type='LEFT'
#         ), 
#         df_args = dict(mapping_to_ami={'premise_nbs':'premise_nbs'}, 
#                        is_df_consolidated=True, 
#                        addtnl_groupby_cols=['trsf_pole_nb']), 
#         field_to_split='df_mp_no_outg', 
#         field_to_split_location_in_kwargs=['df_mp_no_outg'], 
#         sort_coll_to_split=True,
#         batch_size=batch_size, verbose=verbose, n_update=n_update
#     )
#     addtnl_end_events_sql_function_kwargs = dict(
#         build_sql_function_kwargs=dict(opco='oh')
#     )
#     end_events_sql_function_kwargs = {**end_events_sql_function_kwargs, 
#                                       **addtnl_end_events_sql_function_kwargs}
#     #-------------------------
#     start=time.time()
#     end_events = AMIEndEvents(
#         df_construct_type=df_construct_type, 
#         contstruct_df_args = contstruct_df_args_end_events, 
#         build_sql_function=AMIEndEvents_SQL.build_sql_end_events_for_no_outages, 
#         build_sql_function_kwargs=end_events_sql_function_kwargs, 
#         init_df_in_constructor=True, 
#         save_args=save_args
#     )
#     end_events_build_time = time.time()-start

In [15]:
# if not run_using_slim:
#     df_construct_type=DFConstructType.kRunSqlQuery
#     contstruct_df_args_end_events=None
#     build_sql_function = AMIEndEvents_SQL.build_sql_end_events_for_no_outages
#     build_sql_function_kwargs = dict(
#         cols_of_interest=cols_of_interest_end_dev_event,  
#         df_mp_no_outg=df_mp_no_outg, 
#         build_sql_function_kwargs = dict(
#             serialnumber_col='serialnumber', 
#             from_table_alias='un_rin', 
#             schema_name='meter_events', 
#             table_name='end_device_event', 
#             datetime_col = 'valuesinterval', 
#             datetime_pattern = r"([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*", 
#             date_col     = 'aep_event_dt', 
#         ),  
#         split_to_CTEs=True, 
#         max_n_prem_per_outg=1000, 
#         join_mp_args=dict(
#             join_with_CTE=True, 
#             build_mp_kwargs=dict(cols_of_interest=cols_of_interest_met_prem), 
#             join_type='LEFT'
#         ), 
#         #df_args={}, 
#         df_args=dict(addtnl_groupby_cols=['trsf_pole_nb']), 
#         field_to_split='df_mp_no_outg', 
#         field_to_split_location_in_kwargs=['df_mp_no_outg'], 
#         sort_coll_to_split=False,
#         batch_size=10, verbose=True, n_update=1

#     )
#     #-------------------------
#     start=time.time()
#     end_events = AMIEndEvents(
#         df_construct_type=df_construct_type, 
#         contstruct_df_args = contstruct_df_args_end_events, 
#         build_sql_function=build_sql_function, 
#         build_sql_function_kwargs=build_sql_function_kwargs, 
#         init_df_in_constructor=True, 
#         save_args=save_args
#     )
#     end_events_build_time = time.time()-start

In [ ]:
# run_testing = False
# assert(run_testing)

# NOTE: Not really a need to run these calls here, as ultimately I'm using premise numbers, which shouldn't change nearly as much as serial numbers (which already don't change much)

In [ ]:
# def get_historic_SNs_for_PNs(
#     PNs, 
#     df_mp_curr, 
#     df_mp_hist, 
#     assume_one_xfmr_per_PN=True, 
#     output_index=['trsf_pole_nb', 'prem_nb'],
#     assert_all_PNs_found=True, 
#     df_mp_serial_number_col='mfr_devc_ser_nbr', 
#     df_mp_prem_nb_col='prem_nb', 
#     df_mp_install_time_col='inst_ts', 
#     df_mp_removal_time_col='rmvl_ts', 
#     df_mp_trsf_pole_nb_col='trsf_pole_nb'
# ):
#     r"""
#     NOTE: Both df_mp_curr and df_mp_hist are needed because meter_premise_hist does not seem to be completely up-to-date.
#           Therefore, one should actually use the current meter_premise together with the historical
          
#     PNs:
#       Designed to be a list of premise numbers, but a single premise number will work as well
    
#     df_mp_curr:
#       Current meter premise DF, taken from default.meter_premise
      
#     df_mp_hist:
#       Historical meter premise DF, taken from default.meter_premise_hist
      
#     output_index:
#       Columns to use as index for the output.  If None, the DF will be returned just as essentially df_mp_hist joined
#         with df_mp_curr
      
#     assume_one_xfmr_per_PN:
#       The historical data are lacking the transformer number information.  If assume_one_xfmr_per_PN is set to True,
#         any missing transformer numbers for historical entries will be filled with the transformer number inferred from
#         the current meter premise data.
#     """
#     #-------------------------
#     assert(Utilities.is_object_one_of_types(PNs, [list, str, int]))
#     if not isinstance(PNs, list):
#         PNs=[PNs]
#     #-------------------------
#     necessary_mp_cols = [df_mp_serial_number_col, df_mp_prem_nb_col, df_mp_install_time_col, df_mp_removal_time_col]
#     # If df_mp_curr or df_mp_hist are not supplied, they will be built
#     if df_mp_hist is None:
#         mp_hist = MeterPremise(
#             df_construct_type=DFConstructType.kRunSqlQuery, 
#             init_df_in_constructor=True, 
#             build_sql_function=MeterPremise.build_sql_meter_premise, 
#             build_sql_function_kwargs=dict(
#                 cols_of_interest=necessary_mp_cols, 
#                 premise_nbs=PNs, 
#                 table_name='meter_premise_hist'
#             )
#         )
#         df_mp_hist = mp_hist.df
#     #-----
#     if df_mp_curr is None:
#         mp_curr = MeterPremise(
#             df_construct_type=DFConstructType.kRunSqlQuery, 
#             init_df_in_constructor=True, 
#             build_sql_function=MeterPremise.build_sql_meter_premise, 
#             build_sql_function_kwargs=dict(
#                 cols_of_interest=necessary_mp_cols, 
#                 premise_nbs=PNs, 
#                 table_name='meter_premise'
#             )
#         )
#         df_mp_curr = mp_curr.df
#     #-------------------------
#     # At a bare minimum, df_mp_curr and df_mp_hist must both have the following columns:
#     #   necessary_mp_cols = ['mfr_devc_ser_nbr', 'prem_nb', 'inst_ts', 'rmvl_ts']
#     assert(all([x in df_mp_curr.columns for x in necessary_mp_cols]))
#     assert(all([x in df_mp_hist.columns for x in necessary_mp_cols]))
#     if assert_all_PNs_found:
#         assert(len(set(PNs).difference(
#             set(df_mp_curr[df_mp_prem_nb_col]).union(set(df_mp_hist[df_mp_prem_nb_col]))
#         ))==0)
#     #-------------------------
#     # Make sure all dates are datetime object, not e.g., strings
#     if(not is_datetime64_dtype(df_mp_hist[df_mp_install_time_col]) or 
#        not is_datetime64_dtype(df_mp_hist[df_mp_removal_time_col])):
#         # If one isn't, chances are both are not (and no harm in converting both either way)
#         df_mp_hist = Utilities_df.convert_col_types(
#             df=df_mp_hist, 
#             cols_and_types_dict={
#                 df_mp_install_time_col:datetime.datetime, 
#                 df_mp_removal_time_col:datetime.datetime
#             }
#         )
#     #-----
#     if(not is_datetime64_dtype(df_mp_curr[df_mp_install_time_col]) 
#        or not is_datetime64_dtype(df_mp_curr[df_mp_removal_time_col])):
#         # If one isn't, chances are both are not (and no harm in converting both either way)
#         df_mp_curr = Utilities_df.convert_col_types(
#             df=df_mp_curr, 
#             cols_and_types_dict={
#                 df_mp_install_time_col:datetime.datetime, 
#                 df_mp_removal_time_col:datetime.datetime
#             }
#         )    
#     #-------------------------
#     # Make sure the premise numbers in PNs, df_mp_hist, and df_mp_curr are all of the same type
#     #  NOTE: Only checking the first element of each to save time, so not a thorough check but should
#     #        still perform as intended.  Also note, cannot do, e.g., df_mp_hist[df_mp_prem_nb_col].dtype,
#     #        because when elements are strings this will come back as dtype('O'), not str!
#     assert(
#         type(df_mp_hist.iloc[0][df_mp_prem_nb_col])==
#         type(df_mp_curr.iloc[0][df_mp_prem_nb_col])==
#         type(PNs[0])
#     )
#     #-------------------------
#     # From df_mp_curr/df_mp_hist, grab only the PNs needed
#     df_mp_hist = df_mp_hist[df_mp_hist[df_mp_prem_nb_col].isin(PNs)]
#     df_mp_curr = df_mp_curr[df_mp_curr[df_mp_prem_nb_col].isin(PNs)]
    
#     #-------------------------
#     # Combine together df_mp_hist and df_mp_curr.
#     # This can be done either with concat or with a merge.  I'm not sure if one is advantageous over the other,
#     #   but in my limited testing, it appears the concat method is faster (and, in my testing, the results were 
#     #   consistent after the drop_duplicates call)
#     #   I'll leave the merge method commented below, in case further testing is needed/desired
#     # Note: a single serial number can have multiple inst_ts and rmvl_ts entries, therefore these must be included in
#     #       drop_duplicates below (excluding them would elmininate all but one of the service time periods)
#     # Concat method
#     df_mp_hist = pd.concat([df_mp_curr, df_mp_hist])
#     df_mp_hist = df_mp_hist.drop_duplicates(subset=necessary_mp_cols)
    
#     # Merge method
# #     df_mp_hist = pd.merge(
# #         df_mp_curr, 
# #         df_mp_hist, 
# #         how='outer', 
# #         left_on=necessary_mp_cols, 
# #         right_on=necessary_mp_cols
# #     )
# #     df_mp_hist = df_mp_hist.drop_duplicates(subset=necessary_mp_cols)

#     #-------------------------
#     # Fill in missing trsf_pole_nb entries (from the historical meter premise data, which are lacking this column)
#     if assume_one_xfmr_per_PN:
#         # By default, dropna=True in groupby, therefore prem_nbs with NaN trsf_pole_nb (i.e., those for which we are trying to
#         #   fill), will be excluded here, which is the desired functionality.
#         #   Note: I have it explicitly set to True below, but that is not necessary since that's the default
#         trsf_pole_to_prem_nbs = df_mp_hist.groupby(df_mp_trsf_pole_nb_col, dropna=True)[df_mp_prem_nb_col].apply(lambda x: list(set(x)))
        
#         # Build the inverse of trsf_pole_to_prem_nbs
#         prem_nb_to_trsf_pole = {}
#         for trsf_pole_nb, prem_nbs in trsf_pole_to_prem_nbs.items():
#             for prem_nb in prem_nbs:
#                 # Can call assert below because dropna=True in groupby above.
#                 # If one, for whatever reason, wanted to set dropna=False in groupby above,
#                 #   then the if statement below would need to be uncommented
#                 #   Note: No harm in leaving it uncommented regardless
#                 #if pd.isna(k):
#                 #    continue
#                 # The assert assures that each prem_nb only has one (non-Nan) transformer number
#                 # This is important, as it allows me to simplify setting the trsf_pole_nb for NaN values
#                 # below without needing to check this!
#                 assert(prem_nb not in prem_nb_to_trsf_pole)
#                 prem_nb_to_trsf_pole[prem_nb] = trsf_pole_nb
                
#         # Convert prem_nb_to_trsf_pole from dict to Series to allow merge operation
#         prem_nb_to_trsf_pole = pd.Series(prem_nb_to_trsf_pole, name=df_mp_trsf_pole_nb_col)
#         prem_nb_to_trsf_pole.index.name = df_mp_prem_nb_col
        
#         # Take trsf_pole_nb from prem_nb_to_trsf_pole, hence why this column is excluded form df_mp_hist
#         original_column_order = df_mp_hist.columns
#         df_mp_hist = pd.merge(
#             df_mp_hist[[x for x in df_mp_hist.columns if x!=df_mp_trsf_pole_nb_col]], 
#             prem_nb_to_trsf_pole, 
#             left_on=df_mp_prem_nb_col, 
#             right_index=True, 
#             how='left'
#         )
#         df_mp_hist = df_mp_hist[original_column_order] # maintain original order
#     #-------------------------
#     # Set index, if specified
#     if output_index:
#         df_mp_hist = df_mp_hist.set_index(output_index).sort_index()
#     #-------------------------
#     return df_mp_hist



# def get_historic_SNs_for_PNs_in_df(
#     df, 
#     df_mp_curr, 
#     df_mp_hist, 
#     assume_one_xfmr_per_PN=True, 
#     output_index=['trsf_pole_nb', 'prem_nb'],
#     assert_all_PNs_found=True, 
#     df_prem_nb_col='prem_nb', 
#     df_mp_serial_number_col='mfr_devc_ser_nbr', 
#     df_mp_prem_nb_col='prem_nb', 
#     df_mp_install_time_col='inst_ts', 
#     df_mp_removal_time_col='rmvl_ts', 
#     df_mp_trsf_pole_nb_col='trsf_pole_nb'
# ):
#     r"""
#     NOTE: Both df_mp_curr and df_mp_hist are needed because meter_premise_hist does not seem to be completely up-to-date.
#           Therefore, one should actually use the current meter_premise together with the historical
    
#     df_mp_curr:
#       Current meter premise DF, taken from default.meter_premise
      
#     df_mp_hist:
#       Historical meter premise DF, taken from default.meter_premise_hist
      
#     output_index:
#       Columns to use as index for the output.  If None, the DF will be returned just as essentially df_mp_hist joined
#         with df_mp_curr
      
#     assume_one_xfmr_per_PN:
#       The historical data are lacking the transformer number information.  If assume_one_xfmr_per_PN is set to True,
#         any missing transformer numbers for historical entries will be filled with the transformer number inferred from
#         the current meter premise data.
#     """
#     #-------------------------
#     # Make sure the premise numbers in df, df_mp_hist, and df_mp_curr are all of the same type (if the latter two are not None)
#     if df_mp_curr is not None:
#         assert(df[df_prem_nb_col].dtype==df_mp_curr[df_mp_prem_nb_col].dtype)
#     if df_mp_hist is not None:
#         assert(df[df_prem_nb_col].dtype==df_mp_hist[df_mp_prem_nb_col].dtype)
#     #-------------------------
#     # Grab PNs from df and feed everything into get_historic_SNs_for_PNs
#     PNs = df[df_prem_nb_col].unique().tolist()
#     #-------------------------
#     SNs_for_PNs = get_historic_SNs_for_PNs(
#         PNs=PNs, 
#         df_mp_curr=df_mp_curr, 
#         df_mp_hist=df_mp_hist, 
#         assume_one_xfmr_per_PN=assume_one_xfmr_per_PN, 
#         output_index=output_index,
#         assert_all_PNs_found=assert_all_PNs_found, 
#         df_mp_serial_number_col=df_mp_serial_number_col, 
#         df_mp_prem_nb_col=df_mp_prem_nb_col, 
#         df_mp_install_time_col=df_mp_install_time_col, 
#         df_mp_removal_time_col=df_mp_removal_time_col, 
#         df_mp_trsf_pole_nb_col=df_mp_trsf_pole_nb_col
#     )
#     return SNs_for_PNs


# def get_active_SNs_for_PNs_at_datetime_interval(
#     PNs, 
#     df_mp_curr, 
#     df_mp_hist, 
#     dt_0, 
#     dt_1, 
#     assume_one_xfmr_per_PN=True, 
#     output_index=['trsf_pole_nb', 'prem_nb'],
#     output_groupby=None, 
#     include_prems_wo_active_SNs_when_groupby=True, 
#     assert_all_PNs_found=True, 
#     df_mp_serial_number_col='mfr_devc_ser_nbr', 
#     df_mp_prem_nb_col='prem_nb', 
#     df_mp_install_time_col='inst_ts', 
#     df_mp_removal_time_col='rmvl_ts', 
#     df_mp_trsf_pole_nb_col='trsf_pole_nb'
# ):
#     r"""
#     NOTE: Both df_mp_curr and df_mp_hist are needed because meter_premise_hist does not seem to be completely up-to-date.
#           Therefore, one should actually use the current meter_premise together with the historical
    
#     df_mp_curr:
#       Current meter premise DF, taken from default.meter_premise
      
#     df_mp_hist:
#       Historical meter premise DF, taken from default.meter_premise_hist

#     dt_0:
#       Lower limit datetime
#     dt_1:
#       Upper limit datetime
      
#     output_index:
#       Columns to use as index for the output.  If None, the DF will be returned just as essentially df_mp_hist joined
#         with df_mp_curr
        
#     output_groupby:
#       If None, the DF will be returned just as essentially df_mp_hist joined with df_mp_curr.
#       If not None, this should be a column or list of columns.  In this case, the DF which is returned will only include
#         the output_groupby as index (or indices, if multiple), and single column of all serial numbers for each group
#         collected in a list.  The return object will be a pd.Series.
#         i.e., it will return active_SNs_at_time.groupby(output_groupby)[df_mp_serial_number_col].apply(lambda x: list(set(x)))
        
#     include_prems_wo_active_SNs_when_groupby:
#       Only has an effect when output_groupby is not None.  So, for description below, assume that to be true.
#       If False:
#         Premise numbers without any active SNs at the time will be left out of returned series
#       If True:
#         Premise numbers without any active SNs at the time will be included, and their value will be empty lists.
      
#     assume_one_xfmr_per_PN:
#       The historical data are lacking the transformer number information.  If assume_one_xfmr_per_PN is set to True,
#         any missing transformer numbers for historical entries will be filled with the transformer number inferred from
#         the current meter premise data.
#     """
#     #-------------------------
#     # Make sure all dates are datetime object, not e.g., strings
#     dt_0 = pd.to_datetime(dt_0)
#     dt_1 = pd.to_datetime(dt_1)
#     #-------------------------
#     # Build df_mp_hist
#     df_mp_hist = get_historic_SNs_for_PNs(
#         PNs=PNs, 
#         df_mp_curr=df_mp_curr, 
#         df_mp_hist=df_mp_hist, 
#         assume_one_xfmr_per_PN=assume_one_xfmr_per_PN, 
#         output_index=None,
#         assert_all_PNs_found=assert_all_PNs_found, 
#         df_mp_serial_number_col=df_mp_serial_number_col, 
#         df_mp_prem_nb_col=df_mp_prem_nb_col, 
#         df_mp_install_time_col=df_mp_install_time_col, 
#         df_mp_removal_time_col=df_mp_removal_time_col, 
#         df_mp_trsf_pole_nb_col=df_mp_trsf_pole_nb_col
#     )
#     #-------------------------
#     # Enforce the time criteria
#     active_SNs_at_time = df_mp_hist[(df_mp_hist[df_mp_install_time_col]<=pd.to_datetime(date_0)) & 
#                                     (df_mp_hist[df_mp_removal_time_col].fillna(pd.Timestamp.max)>pd.to_datetime(date_1))]
#     #-------------------------
#     if include_prems_wo_active_SNs_when_groupby and output_groupby is not None:
#         prems_wo_active_SNs = list(set(PNs).difference(set(active_SNs_at_time[df_mp_prem_nb_col])))
#         df_mp_hist_wo = df_mp_hist[df_mp_hist[df_mp_prem_nb_col].isin(prems_wo_active_SNs)]
#         # Below line simply makes a pd.Series object with index (indices) equal to output_groupby values and values
#         #   equal to empty lists. This is much quicker than my old method.
#         prems_wo_active_SNs = df_mp_hist_wo.groupby(output_groupby)[df_mp_serial_number_col].agg(lambda x: [])
#     #-------------------------
#     # Set index, if specified
#     if output_index:
#         active_SNs_at_time = active_SNs_at_time.set_index(output_index).sort_index()
#     #-------------------------
#     # Groupby, if specified
#     # NOTE: In this case, the DF which is returned will only include the output_groupby as index (or indices, if multiple), 
#     #       and single column of all serial numbers for each group collected in a list.
#     #       The return object will be a pd.Series
#     if output_groupby:
#         active_SNs_at_time = active_SNs_at_time.groupby(output_groupby)[df_mp_serial_number_col].apply(lambda x: sorted(list(set(x))))
#         if include_prems_wo_active_SNs_when_groupby:
#             # Depending on what columns are used for output_groupby, it is possible that an index is found in both
#             #   active_SNs_at_time and prems_wo_active_SNs.  For instance, if grouping by only trsf_pole_nb, if some of
#             #   the SNs were active, but others were not, then the index would be in both.  In this case, in the final
#             #   pd.Series, we don't want then entry from prems_wo_active_SNs (if included, there would be an entry for
#             #   the index with the SNs included, and an entry with an empty list)
#             prems_wo_active_SNs = prems_wo_active_SNs.loc[~prems_wo_active_SNs.index.isin(active_SNs_at_time.index)] 
#             active_SNs_at_time = pd.concat([active_SNs_at_time, prems_wo_active_SNs])
#             active_SNs_at_time = active_SNs_at_time.sort_index()
#     #-------------------------
#     return active_SNs_at_time


# def get_active_SNs_for_PNs_in_df_at_datetime_interval(
#     df, 
#     df_mp_curr, 
#     df_mp_hist, 
#     dt_0, 
#     dt_1, 
#     assume_one_xfmr_per_PN=True, 
#     output_index=['trsf_pole_nb', 'prem_nb'],
#     output_groupby=None, 
#     include_prems_wo_active_SNs_when_groupby=True, 
#     assert_all_PNs_found=True, 
#     df_prem_nb_col='prem_nb', 
#     df_mp_serial_number_col='mfr_devc_ser_nbr', 
#     df_mp_prem_nb_col='prem_nb', 
#     df_mp_install_time_col='inst_ts', 
#     df_mp_removal_time_col='rmvl_ts', 
#     df_mp_trsf_pole_nb_col='trsf_pole_nb'
# ):
#     r"""
#     NOTE: Both df_mp_curr and df_mp_hist are needed because meter_premise_hist does not seem to be completely up-to-date.
#           Therefore, one should actually use the current meter_premise together with the historical
    
#     df_mp_curr:
#       Current meter premise DF, taken from default.meter_premise
      
#     df_mp_hist:
#       Historical meter premise DF, taken from default.meter_premise_hist

#     dt_0:
#       Lower limit datetime
#     dt_1:
#       Upper limit datetime
      
#     output_index:
#       Columns to use as index for the output.  If None, the DF will be returned just as essentially df_mp_hist joined
#         with df_mp_curr
        
#     output_groupby:
#       If None, the DF will be returned just as essentially df_mp_hist joined with df_mp_curr.
#       If not None, this should be a column or list of columns.  In this case, the DF which is returned will only include
#         the output_groupby as index (or indices, if multiple), and single column of all serial numbers for each group
#         collected in a list.  The return object will be a pd.Series.
#         i.e., it will return active_SNs_at_time.groupby(output_groupby)[df_mp_serial_number_col].apply(lambda x: list(set(x)))
        
#     include_prems_wo_active_SNs_when_groupby:
#       Only has an effect when output_groupby is not None.  So, for description below, assume that to be true.
#       If False:
#         Premise numbers without any active SNs at the time will be left out of returned series
#       If True:
#         Premise numbers without any active SNs at the time will be included, and their value will be empty lists.
      
#     assume_one_xfmr_per_PN:
#       The historical data are lacking the transformer number information.  If assume_one_xfmr_per_PN is set to True,
#         any missing transformer numbers for historical entries will be filled with the transformer number inferred from
#         the current meter premise data.
#     """
#     #-------------------------
#     # Grab PNs from df and feed everything into get_historic_SNs_for_PNs
#     PNs = df[df_prem_nb_col].unique().tolist()
#     #-------------------------
#     active_SNs_for_PNs_at_dt = get_active_SNs_for_PNs_at_datetime_interval(
#         PNs=PNs, 
#         df_mp_curr=df_mp_curr, 
#         df_mp_hist=df_mp_hist, 
#         dt_0=dt_0, 
#         dt_1=dt_1, 
#         assume_one_xfmr_per_PN=assume_one_xfmr_per_PN, 
#         output_index=output_index,
#         output_groupby=output_groupby, 
#         include_prems_wo_active_SNs_when_groupby=include_prems_wo_active_SNs_when_groupby, 
#         assert_all_PNs_found=assert_all_PNs_found, 
#         df_mp_serial_number_col=df_mp_serial_number_col, 
#         df_mp_prem_nb_col=df_mp_prem_nb_col, 
#         df_mp_install_time_col=df_mp_install_time_col, 
#         df_mp_removal_time_col=df_mp_removal_time_col, 
#         df_mp_trsf_pole_nb_col=df_mp_trsf_pole_nb_col
#     )
#     return active_SNs_for_PNs_at_dt


# def get_active_SNs_for_PN_at_datetime_interval(
#     PN, 
#     df_mp_curr, 
#     df_mp_hist, 
#     dt_0, 
#     dt_1, 
#     df_mp_serial_number_col='mfr_devc_ser_nbr', 
#     df_mp_prem_nb_col='prem_nb', 
#     df_mp_install_time_col='inst_ts', 
#     df_mp_removal_time_col='rmvl_ts'
# ):
#     r"""
#     PN:
#       Premise number
#     df_mp_hist:
#       meter_premise_hist DF
#     dt_0:
#       Lower limit datetime
#     dt_1:
#       Upper limit datetime
#     """
#     #-------------------------
#     SNs_for_PN_at_dt = get_active_SNs_for_PNs_at_datetime_interval(
#         PNs=PN, 
#         df_mp_curr=df_mp_curr, 
#         df_mp_hist=df_mp_hist, 
#         dt_0=dt_0, 
#         dt_1=dt_1, 
#         assume_one_xfmr_per_PN=False, 
#         output_index=None,
#         output_groupby=None, 
#         include_prems_wo_active_SNs_when_groupby=False, 
#         assert_all_PNs_found=True, 
#         df_mp_serial_number_col=df_mp_serial_number_col, 
#         df_mp_prem_nb_col=df_mp_prem_nb_col, 
#         df_mp_install_time_col=df_mp_install_time_col, 
#         df_mp_removal_time_col=df_mp_removal_time_col
#     )
#     #-------------------------
#     SNs_for_PN_at_dt = SNs_for_PN_at_dt[df_mp_serial_number_col].unique().tolist()
#     return SNs_for_PN_at_dt

# def get_active_SNs_for_PN_at_datetime(
#     PN, 
#     df_mp_curr, 
#     df_mp_hist, 
#     dt_0, 
#     df_mp_serial_number_col='mfr_devc_ser_nbr', 
#     df_mp_prem_nb_col='prem_nb', 
#     df_mp_install_time_col='inst_ts', 
#     df_mp_removal_time_col='rmvl_ts'
# ):
#     r"""
#     prem_nb:
#       Premise number
#     df_mp_hist:
#       meter_premise_hist DF
#     dt_0:
#       Datetime
#     """
#     #-------------------------
#     return get_active_SNs_for_PN_at_datetime_interval(
#         PN=PN, 
#         df_mp_curr=df_mp_curr, 
#         df_mp_hist=df_mp_hist, 
#         dt_0=dt_0, 
#         dt_1=dt_0, 
#         df_mp_serial_number_col=df_mp_serial_number_col, 
#         df_mp_prem_nb_col=df_mp_prem_nb_col, 
#         df_mp_install_time_col=df_mp_install_time_col, 
#         df_mp_removal_time_col=df_mp_removal_time_col
#     )

In [16]:
df_outage_location_ids_OG = pd.read_csv(os.path.join(save_dir_csvs, 'df_outage_location_ids_OG.csv'), dtype=str)
df_mp_all_OG = pd.read_csv(os.path.join(save_dir_csvs, 'df_mp_all_OG.csv'), dtype=str)
df_mp_no_outg_OG = pd.read_csv(os.path.join(save_dir_csvs, 'df_mp_no_outg_OG.csv'), dtype=str)
df_mp_outg_OG = pd.read_csv(os.path.join(save_dir_csvs, 'df_mp_outg_OG.csv'), dtype=str)

df_mp_no_outg_slim_OG = read_consolidated_df_from_csv(
    file_path=os.path.join(save_dir_csvs, 'df_mp_no_outg_slim_OG.csv'), 
    list_cols = ['serial_numbers', 'premise_nbs']
)

In [17]:
print(f'df_outage_location_ids_OG.shape: {df_outage_location_ids_OG.shape}')
print(f'df_mp_all_OG.shape: {df_mp_all_OG.shape}')
print(f'df_mp_outg_OG.shape: {df_mp_outg_OG.shape}')
print(f'df_mp_no_outg_OG.shape: {df_mp_no_outg_OG.shape}')
#-------------------------
df_outage_location_ids = df_outage_location_ids_OG.copy()
#----------
df_mp_all_OG = clm.remove_prepend_from_columns_in_df(df_mp_all_OG)
df_mp_all = df_mp_all_OG.copy()
#----------
df_mp_no_outg = df_mp_no_outg_OG.copy()
df_mp_no_outg = Utilities_df.convert_col_types(
    df=df_mp_no_outg, 
    cols_and_types_dict={'annual_kwh':float, 'annual_max_dmnd':float}, 
    to_numeric_errors='coerce', 
    inplace=True
)
#----------
df_mp_outg = df_mp_outg_OG.copy()
#----------
print(f'df_mp_no_outg_slim_OG.shape: {df_mp_no_outg_slim_OG.shape}')
df_mp_no_outg_slim = df_mp_no_outg_slim_OG.copy()

df_outage_location_ids_OG.shape: (13635, 1)
df_mp_all_OG.shape: (1544705, 21)
df_mp_outg_OG.shape: (42745, 21)
df_mp_no_outg_OG.shape: (1033129, 23)
df_mp_no_outg_slim_OG.shape: (192661, 11)


In [18]:
start=time.time()
mp_hist = MeterPremise(
    df_construct_type=DFConstructType.kRunSqlQuery, 
    init_df_in_constructor=True, 
    build_sql_function=MeterPremise.build_sql_meter_premise, 
    build_sql_function_kwargs=dict(
        cols_of_interest=[
            'mfr_devc_ser_nbr',
            'prem_nb',    
            'inst_ts', 
            'rmvl_ts'            
        ], 
        states=['OH'], 
        table_name='meter_premise_hist'
    )
)
print(time.time()-start)

df_mp_hist_all = mp_hist.df.copy()
df_mp_hist_all = Utilities_df.convert_col_types(
    df=df_mp_hist_all, 
    cols_and_types_dict={
        'inst_ts':datetime.datetime, 
        'rmvl_ts':datetime.datetime
    }
)
df_mp_hist_all_OG = df_mp_hist_all.copy()

238.87240362167358


In [19]:
# meter_premise_hist does not seem to be completely up-to-date.
# Therefore, one should actually use the current meter_premise together with the historical

In [20]:
start=time.time()
mp_curr = MeterPremise(
    df_construct_type=DFConstructType.kRunSqlQuery, 
    init_df_in_constructor=True, 
    build_sql_function=MeterPremise.build_sql_meter_premise, 
    build_sql_function_kwargs=dict(
#         cols_of_interest=[
#             'mfr_devc_ser_nbr',
#             'prem_nb',    
#             'inst_ts', 
#             'rmvl_ts'            
#         ], 
        cols_of_interest=cols_of_interest_met_prem+['inst_ts', 'rmvl_ts'], 
        states=['OH'], 
        table_name='meter_premise'
    )
)
print(time.time()-start)

df_mp_curr_all = mp_curr.df.copy()
df_mp_curr_all = Utilities_df.convert_col_types(
    df=df_mp_curr_all, 
    cols_and_types_dict={
        'inst_ts':datetime.datetime, 
        'rmvl_ts':datetime.datetime
    }
)
df_mp_curr_all_OG = df_mp_curr_all.copy()

162.00002598762512


In [21]:
import MeterPremise
reload(MeterPremise)
from MeterPremise import MeterPremise

In [22]:
# date_0 = '2022-07-01'
# date_1 = '2022-07-15'

In [23]:
start=time.time()
test_vals = MeterPremise.get_active_SNs_for_PN_at_datetime_interval('100560751', df_mp_curr_all, df_mp_hist_all, dt_0=date_0, dt_1=date_1)
print(time.time()-start)
test_vals

1.7209324836730957


['881984622']

In [24]:
df_mp_curr_all = df_mp_curr_all_OG.copy()
df_mp_hist_all = df_mp_hist_all_OG.copy()

In [25]:
start=time.time()
ressy = MeterPremise.get_active_SNs_for_PNs_in_df_at_datetime_interval(
    df=df_mp_all, 
    df_mp_curr=df_mp_curr_all, 
    df_mp_hist=df_mp_hist_all, 
    dt_0=date_0, 
    dt_1=date_1, 
    assume_one_xfmr_per_PN=True, 
    output_index=['trsf_pole_nb', 'prem_nb'],
    output_groupby=['trsf_pole_nb', 'prem_nb'], 
    include_prems_wo_active_SNs_when_groupby=True
)
print(time.time()-start)

AssertionError: 

In [ ]:
ressy.loc[['1680442441338', '1680532437815', 'TRANSMISSION']]

In [64]:
MeterPremise.get_active_SNs_for_PN_at_datetime_interval(
    '077839970',
    df_mp_curr_all, 
    df_mp_hist_all, 
    date_0,
    date_1
)

[]

In [65]:
start=time.time()
ugh = MeterPremise.get_historic_SNs_for_PNs_in_df(
    df=df_mp_all, 
#     df=df_mp_all[df_mp_all['trsf_pole_nb'].isin(['1680442441338', '1680532437815', 'TRANSMISSION'])], 
    df_mp_curr=df_mp_curr_all, 
    df_mp_hist=df_mp_hist_all, 
    assume_one_xfmr_per_PN=True, 
    output_index=['trsf_pole_nb', 'prem_nb']
)
print(time.time()-start)

AssertionError: 

In [ ]:
ugh.index

In [ ]:
ugh

In [67]:
idx = pd.IndexSlice
ugh.loc[idx[:, '077839970'], :]

NameError: name 'ugh' is not defined

In [68]:
ugh.loc[(slice(None), '077839970'), :]

NameError: name 'ugh' is not defined

In [69]:
# import MeterPremise
# reload(MeterPremise)
# from MeterPremise import MeterPremise

In [70]:
trsf_pole_nbs = df_mp_curr_all['trsf_pole_nb'].unique().tolist()[:10]

In [71]:
len(trsf_pole_nbs)

10

In [72]:
start=time.time()
sns_for_xfrms = MeterPremise.get_SNs_for_xfmrs(df_mp_curr_all['trsf_pole_nb'].unique().tolist()[:1000], mp_df=df_mp_curr_all_OG.copy())
print(time.time()-start)

0.7392823696136475


In [73]:
df_mp_curr = df_mp_curr_all_OG.copy()
df_mp_hist = df_mp_hist_all_OG.copy()

In [74]:
start=time.time()
SNs_and_PNs_for_xfrms, df_mp_curr = MeterPremise.get_SNs_for_xfmrs(
    trsf_pole_nbs=df_mp_curr_all['trsf_pole_nb'].unique().tolist(), 
    include_prem_nbs=True,
    assert_all_trsf_pole_nbs_found=True,        
    mp_df=df_mp_curr, 
    return_mp_df_also=True
)
print(time.time()-start)

6.588074445724487


In [75]:
SNs_and_PNs_for_xfrms.equals(sns_for_xfrms)

False

In [76]:
trsfs = df_mp_curr_all['trsf_pole_nb'].unique().tolist()[:1000]
PNs = df_mp_curr[df_mp_curr['trsf_pole_nb'].isin(trsfs)]['prem_nb'].unique().tolist()

In [77]:
len(PNs)

2040

In [78]:
start=time.time()
PNs2 = Utilities_df.consolidate_column_of_lists(sns_for_xfrms, 'prem_nbs')
print(time.time()-start)

0.004985809326171875


In [79]:
len(PNs2)

2040

In [80]:
sorted(PNs)==sorted(PNs2)

True

In [ ]:
# PNs

In [ ]:
# @staticmethod
# def get_SNs_for_xfmrs(
#     trsf_pole_nbs,
#     include_prem_nbs=True,
#     trsf_pole_nb_col='trsf_pole_nb', 
#     serial_number_col='mfr_devc_ser_nbr', 
#     prem_nb_col='prem_nb', 
#     return_SNs_col='SNs',
#     return_prem_nbs_col='prem_nbs'
# ):
        
        
        
        
# def get_historic_SNs_for_PNs(
#     PNs, 
#     df_mp_curr, 
#     df_mp_hist, 
#     assume_one_xfmr_per_PN=True, 
#     output_index=['trsf_pole_nb', 'prem_nb'],
#     assert_all_PNs_found=True, 
#     df_mp_serial_number_col='mfr_devc_ser_nbr', 
#     df_mp_prem_nb_col='prem_nb', 
#     df_mp_install_time_col='inst_ts', 
#     df_mp_removal_time_col='rmvl_ts', 
#     df_mp_trsf_pole_nb_col='trsf_pole_nb'
# ):

In [38]:
def get_historic_PNs_and_SNs_for_xfmrs(
    trsf_pole_nbs, 
    df_mp_curr, 
    df_mp_hist,  
    return_consolidated=True, 
    output_index=['trsf_pole_nb', 'prem_nb'],
    output_groupby=None, 
    assert_all_PNs_found=True, 
    assert_all_trsf_pole_nbs_found=True, 
    df_mp_serial_number_col='mfr_devc_ser_nbr', 
    df_mp_prem_nb_col='prem_nb', 
    df_mp_install_time_col='inst_ts', 
    df_mp_removal_time_col='rmvl_ts', 
    df_mp_trsf_pole_nb_col='trsf_pole_nb', 
    consolidated_SNs_col='SNs',
    consolidated_PNs_col='PNs'
):
    r"""
    The main trouble here is that meter_premise_hist does not contain a trsf_pole_nb field.
    Therefore, one must grab the PNs for each transformer from the current meter_premise data,
    and use that information to grab the correct entries from the historic set.
    
    return_consolidated:
      If True, this overrides output_index and output_groupby to return a DF with the same form as 
        that from get_SNs_for_xfmrs, i.e., the DF that will be returned will have indices equal to trsf_pole_nbs
        and a SNs and PNs columns each containing a list of serial numbers and premise numbers, respectively.
    
    See MeterPremise.get_historic_SNs_for_PNs for information about other parameters.
    """
    start=time.time()
    #-------------------------
    necessary_mp_cols = [df_mp_serial_number_col, df_mp_prem_nb_col, df_mp_install_time_col, df_mp_removal_time_col]
    #-------------------------
    print('1')
    print(time.time()-start)
    start=time.time()
    if df_mp_curr is None:
        mp_curr = MeterPremise(
            df_construct_type=DFConstructType.kRunSqlQuery, 
            contstruct_df_args=None, 
            init_df_in_constructor=True, 
            build_sql_function = MeterPremise.build_sql_meter_premise, 
            build_sql_function_kwargs = dict(
                cols_of_interest=necessary_mp_cols,  
                trsf_pole_nbs=trsf_pole_nbs, 
                field_to_split='trsf_pole_nbs', 
                batch_size=15000,
                n_update=1000, 
                table_name='meter_premise'
            )
        )
        df_mp_curr = mp_curr.df
    #-------------------------
    print('2')
    print(time.time()-start)
    start=time.time()
    if assert_all_trsf_pole_nbs_found:
        assert(len(set(trsf_pole_nbs).difference(set(df_mp_curr[df_mp_trsf_pole_nb_col])))==0)
    #-------------------------
    print('3')
    print(time.time()-start)
    start=time.time()
    PNs = df_mp_curr[df_mp_curr[df_mp_trsf_pole_nb_col].isin(trsf_pole_nbs)][df_mp_prem_nb_col].unique().tolist()
    #-------------------------
    print('4')
    print(time.time()-start)
    start=time.time()
    # Now, armed with the PNs, df_mp_hist can be built (if need be)
    if df_mp_hist is None:
        mp_hist = MeterPremise(
            df_construct_type=DFConstructType.kRunSqlQuery, 
            contstruct_df_args=None, 
            init_df_in_constructor=True, 
            build_sql_function = MeterPremise.build_sql_meter_premise, 
            build_sql_function_kwargs = dict(
                cols_of_interest=necessary_mp_cols,  
                premise_nbs=PNs, 
                table_name='meter_premise_hist'
            )
        )
        df_mp_hist = mp_hist.df
    print('5')
    print(time.time()-start)
    start=time.time()
    #-------------------------
    # At a bare minimum, df_mp_curr and df_mp_hist must both have the following columns:
    #   necessary_mp_cols = ['mfr_devc_ser_nbr', 'prem_nb', 'inst_ts', 'rmvl_ts']
    assert(all([x in df_mp_curr.columns for x in necessary_mp_cols]))
    assert(all([x in df_mp_hist.columns for x in necessary_mp_cols])) 
    print('6')
    print(time.time()-start)
    start=time.time()
    #-------------------------
    if return_consolidated:
        output_index = None
        output_groupby = None
    hist_SNs_for_xfmrs = MeterPremise.get_historic_SNs_for_PNs(
        PNs=PNs, 
        df_mp_curr=df_mp_curr, 
        df_mp_hist=df_mp_hist, 
        assume_one_xfmr_per_PN=True, 
        output_index=output_index,
        output_groupby=output_groupby,
        assert_all_PNs_found=True, 
        df_mp_serial_number_col=df_mp_serial_number_col, 
        df_mp_prem_nb_col=df_mp_prem_nb_col, 
        df_mp_install_time_col=df_mp_install_time_col, 
        df_mp_removal_time_col=df_mp_removal_time_col, 
        df_mp_trsf_pole_nb_col=df_mp_trsf_pole_nb_col
    )
    print('7')
    print(time.time()-start)
    start=time.time()
    #-------------------------
    if return_consolidated:
        hist_SNs_for_xfmrs = Utilities_df.consolidate_df(
            df=hist_SNs_for_xfmrs, 
            groupby_col=df_mp_trsf_pole_nb_col, 
            cols_shared_by_group=None, 
            cols_to_collect_in_lists=[df_mp_prem_nb_col, df_mp_serial_number_col], 
            include_groupby_col_in_output_cols=False, 
            allow_duplicates_in_lists=False, 
            recover_uniqueness_violators=True, 
            rename_cols={
                df_mp_serial_number_col:consolidated_SNs_col, 
                df_mp_prem_nb_col:consolidated_PNs_col
            }, 
            verbose=False
        )
    print('8')
    print(time.time()-start)
    start=time.time()
    return hist_SNs_for_xfmrs    

In [39]:
df_mp_curr = df_mp_curr_all_OG.copy()
df_mp_hist = df_mp_hist_all_OG.copy()

In [40]:
PNs = df_mp_all['prem_nb'].unique().tolist()
trsf_pole_nbs = df_mp_all['trsf_pole_nb'].unique().tolist()

In [41]:
start=time.time()
hist_from_PNs = MeterPremise.get_historic_SNs_for_PNs(
    PNs=PNs, 
    df_mp_curr=df_mp_curr, 
    df_mp_hist=df_mp_hist, 
    assume_one_xfmr_per_PN=True, 
    output_index=['trsf_pole_nb', 'prem_nb'],
    output_groupby=['trsf_pole_nb', 'prem_nb']
)
print(time.time()-start)

AssertionError: 

In [42]:
df_mp_curr.equals(df_mp_curr_all_OG.copy())

True

In [43]:
FUCK

NameError: name 'FUCK' is not defined

In [ ]:
len(PNs)

In [44]:
import MeterPremise
reload(MeterPremise)
from MeterPremise import MeterPremise

In [45]:
#TESTING TIME, DELETE ME
start=time.time()
mp_hist = MeterPremise(
    df_construct_type=DFConstructType.kRunSqlQuery, 
    init_df_in_constructor=True, 
    build_sql_function=MeterPremise.build_sql_meter_premise, 
    build_sql_function_kwargs=dict(
        cols_of_interest=[
            'mfr_devc_ser_nbr',
            'prem_nb',    
            'inst_ts', 
            'rmvl_ts'            
        ], 
        premise_nbs=PNs, 
        field_to_split='premise_nbs', 
        batch_size=15000,
        n_update=1000, 
        table_name='meter_premise_hist'
    ), 
    max_n_prem_nbs=len(PNs)+1
)
print(time.time()-start)

n_coll = 1542699
batch_size = 15000
n_batches = 103
2557.6778066158295


In [46]:
start=time.time()
mp_hist_v1 = MeterPremise(
    df_construct_type=DFConstructType.kRunSqlQuery, 
    init_df_in_constructor=True, 
    build_sql_function=MeterPremise.build_sql_meter_premise, 
    build_sql_function_kwargs=dict(
        cols_of_interest=[
            'mfr_devc_ser_nbr',
            'prem_nb',    
            'inst_ts', 
            'rmvl_ts'            
        ], 
        states=['OH'], 
        table_name='meter_premise_hist'
    )
)
print(time.time()-start)

226.20860624313354


In [47]:
mp_hist.df.shape

(4163250, 4)

In [48]:
mp_hist_v1.df.shape

(4495658, 4)

In [49]:
mp_hist_v1.df

,mfr_devc_ser_nbr,prem_nb,inst_ts,rmvl_ts
0,58754969,105825231,1973-02-01 00:00:00,2010-12-08 12:00:00
1,875536731,106912101,2018-03-06 12:00:20,
2,659434695,107932151,2013-12-28 12:00:20,
3,656468929,100376689,2014-02-24 12:00:20,
4,880679324,100863030,2018-09-19 12:00:20,
...,...,...,...,...
4495653,428405808,105377591,2012-09-20 12:00:20,2020-06-06 12:00:00
4495654,428405808,105377591,2012-09-20 12:00:20,2020-06-06 12:00:00
4495655,190879653,108837194,2016-07-01 12:00:20,2018-07-13 12:00:00
4495656,66036563,100999081,1980-02-01 00:00:00,2018-10-18 12:00:00


In [50]:
diff = pd.merge(mp_hist_v1.df, mp_hist.df, how='outer', indicator=True)

In [51]:
diff[diff['_merge']!='both']

,mfr_devc_ser_nbr,prem_nb,inst_ts,rmvl_ts,_merge
79,190773450,104197373,2017-03-27 12:00:20,2017-09-20 12:00:00,left_only
94,16273481,104249975,1999-03-05 12:00:19,2010-01-04 12:00:00,left_only
101,38115290,102585151,1994-06-01 00:00:00,2001-04-18 12:00:00,left_only
102,223434264,101421952,2003-03-24 12:00:20,2004-03-23 12:00:00,left_only
112,445766305,103121590,2014-08-06 12:00:20,2015-03-27 12:00:00,left_only
...,...,...,...,...,...
5472366,535319046,101128017,2008-11-11 12:00:20,2019-11-12 12:00:00,left_only
5472388,64027275,102446610,1978-02-01 00:00:00,2014-03-11 12:00:00,left_only
5472390,44269525,102433562,1964-02-01 00:00:00,2006-08-07 12:00:00,left_only
5472404,226418133,108002460,2006-10-04 12:00:20,2018-10-26 12:00:00,left_only


In [52]:
diff[diff['_merge']=='right_only']

,mfr_devc_ser_nbr,prem_nb,inst_ts,rmvl_ts,_merge


In [53]:
start=time.time()
hist_from_xfmrs = get_historic_PNs_and_SNs_for_xfmrs(
    trsf_pole_nbs=trsf_pole_nbs, 
    df_mp_curr=df_mp_curr, 
    df_mp_hist=None, 
    return_consolidated=False, 
    output_index=['trsf_pole_nb', 'prem_nb'],
    output_groupby=['trsf_pole_nb', 'prem_nb']
)
print(time.time()-start)

1
0.0
2
0.0


AssertionError: 

In [54]:
def get_active_PNs_and_SNs_for_xfmrs_at_datetime_interval(
    trsf_pole_nbs, 
    df_mp_curr, 
    df_mp_hist,  
    dt_0, 
    dt_1, 
    return_consolidated=True, 
    output_index=['trsf_pole_nb', 'prem_nb'],
    output_groupby=None, 
    assert_all_PNs_found=True, 
    df_mp_serial_number_col='mfr_devc_ser_nbr', 
    df_mp_prem_nb_col='prem_nb', 
    df_mp_install_time_col='inst_ts', 
    df_mp_removal_time_col='rmvl_ts', 
    df_mp_trsf_pole_nb_col='trsf_pole_nb', 
    consolidated_SNs_col='SNs',
    consolidated_PNs_col='PNs'
):
    r"""
    The main trouble here is that meter_premise_hist does not contain a trsf_pole_nb field.
    Therefore, one must grab the PNs for each transformer from the current meter_premise data,
    and use that information to grab the correct entries from the historic set.
    
    return_consolidated:
      If True, this overrides output_index and output_groupby to return a DF with the same form as 
        that from get_SNs_for_xfmrs, i.e., the DF that will be returned will have indices equal to trsf_pole_nbs
        and a SNs and PNs columns each containing a list of serial numbers and premise numbers, respectively.
    
    See MeterPremise.get_active_SNs_for_PNs_at_datetime_interval for information about other parameters.
    """
    #-------------------------
    # Make sure all dates are datetime object, not e.g., strings
    dt_0 = pd.to_datetime(dt_0)
    dt_1 = pd.to_datetime(dt_1)
    #-------------------------    
    # Build df_mp_hist
    df_mp_hist = get_historic_PNs_and_SNs_for_xfmrs(
        trsf_pole_nbs=trsf_pole_nbs, 
        df_mp_curr=df_mp_curr, 
        df_mp_hist=df_mp_hist,  
        return_consolidated=False, 
        output_index=None,
        output_groupby=None, 
        assert_all_PNs_found=assert_all_PNs_found, 
        df_mp_serial_number_col=df_mp_serial_number_col, 
        df_mp_prem_nb_col=df_mp_prem_nb_col, 
        df_mp_install_time_col=df_mp_install_time_col, 
        df_mp_removal_time_col=df_mp_removal_time_col, 
        df_mp_trsf_pole_nb_col=df_mp_trsf_pole_nb_col, 
        consolidated_SNs_col=consolidated_SNs_col,
        consolidated_PNs_col=consolidated_PNs_col
    )
    #-------------------------
    # Enforce the time criteria
    active_SNs_at_time = df_mp_hist[(df_mp_hist[df_mp_install_time_col]<=pd.to_datetime(dt_0)) & 
                                    (df_mp_hist[df_mp_removal_time_col].fillna(pd.Timestamp.max)>pd.to_datetime(dt_1))]
    #-------------------------
    if return_consolidated:
        output_index = None
        output_groupby = None
        active_SNs_at_time = Utilities_df.consolidate_df(
            df=active_SNs_at_time, 
            groupby_col=df_mp_trsf_pole_nb_col, 
            cols_shared_by_group=None, 
            cols_to_collect_in_lists=[df_mp_prem_nb_col, df_mp_serial_number_col], 
            include_groupby_col_in_output_cols=False, 
            allow_duplicates_in_lists=False, 
            recover_uniqueness_violators=True, 
            rename_cols=None, 
            verbose=False
        )
    #-------------------------
    # Set index, if specified
    if output_index:
        active_SNs_at_time = active_SNs_at_time.set_index(output_index).sort_index()
    #-------------------------
    # Groupby, if specified
    # NOTE: In this case, the DF which is returned will only include the output_groupby as index (or indices, if multiple), 
    #       and single column of all serial numbers for each group collected in a list.
    #       The return object will be a pd.Series
    if output_groupby:
        active_SNs_at_time = active_SNs_at_time.groupby(output_groupby)[df_mp_serial_number_col].apply(lambda x: sorted(list(set(x))))
    #-------------------------
    return active_SNs_at_time

In [55]:
df_mp_curr_all.head()

,mfr_devc_ser_nbr,longitude,latitude,state_cd,prem_nb,srvc_pole_nb,trsf_pole_nb,latitude_nb,longitude_nb,circuit_nb,...,xfmr_nb,annual_kwh,annual_max_dmnd,mtr_stat_cd,mtr_stat_cd_desc,devc_stat_cd,devc_stat_cd_desc,curr_acct_cls_cd,inst_ts,rmvl_ts
0,300023952,-82.53942764,38.4058079,OH,078036129,38830119D30001,38830119D30001,38.4058079,-82.53942764,,...,,75993.000,0.000,C,CONNECTED,A,ACTIVE,R,2018-10-18 12:00:20,NaT
1,300023953,-82.28310156,38.59069353,OH,073574480,39821038B10018,39821038B10017,38.59069353,-82.28310156,,...,,3102.000,0.000,C,CONNECTED,A,ACTIVE,P,2018-10-17 12:00:20,NaT
2,300023954,-82.85232000,39.031350000,OH,076684265,39830544000129,39830544000129,39.031350000,-82.85232000,,...,,26242.000,0.000,C,CONNECTED,A,ACTIVE,R,2018-10-18 12:00:20,NaT
3,300023959,-82.84455574,38.69047534,OH,074663922,39830928D20281,39830928D20281,38.69047534,-82.84455574,,...,,47585.000,0.000,C,CONNECTED,A,ACTIVE,R,2018-10-22 12:00:20,NaT
4,300023960,-82.64657000,38.536100000,OH,078364867,39831125A40044,39831125A40001,38.536100000,-82.64657000,,...,,22257.000,0.000,C,CONNECTED,A,ACTIVE,R,2018-10-23 12:00:20,NaT


In [56]:
df_mp_hist_all.head()

,mfr_devc_ser_nbr,prem_nb,inst_ts,rmvl_ts
0,58754969,105825231,1973-02-01 00:00:00,2010-12-08 12:00:00
1,875536731,106912101,2018-03-06 12:00:20,NaT
2,659434695,107932151,2013-12-28 12:00:20,NaT
3,656468929,100376689,2014-02-24 12:00:20,NaT
4,880679324,100863030,2018-09-19 12:00:20,NaT


In [57]:
start=time.time()
res_from_PNs = MeterPremise.get_active_SNs_for_PNs_at_datetime_interval(
    PNs=PNs, 
    df_mp_curr=df_mp_curr_all, 
    df_mp_hist=df_mp_hist_all, 
    dt_0=date_0, 
    dt_1=date_1, 
    assume_one_xfmr_per_PN=True, 
    output_index=['trsf_pole_nb', 'prem_nb'],
    output_groupby=['trsf_pole_nb', 'prem_nb'], 
    include_prems_wo_active_SNs_when_groupby=False
)
print(time.time()-start)

AssertionError: 

In [ ]:
res_from_PNs

In [ ]:
start=time.time()
res_from_xfmrs = get_active_PNs_and_SNs_for_xfmrs_at_datetime_interval(
    trsf_pole_nbs=trsf_pole_nbs, 
    df_mp_curr=df_mp_curr_all, 
    df_mp_hist=df_mp_hist_all, 
    dt_0=date_0, 
    dt_1=date_1, 
    output_index=['trsf_pole_nb', 'prem_nb'],
    output_groupby=['trsf_pole_nb', 'prem_nb']
)
print(time.time()-start)

In [58]:
trsf_pole_nbs = df_mp_curr_all['trsf_pole_nb'].unique().tolist()[:10]
df_mp_curr = df_mp_curr_all_OG.copy()
df_mp_hist = df_mp_hist_all_OG.copy()
return_consolidated=True # If True, this overrides output_index and output_groupby to return a DF with the same form as that from get_SNs_for_xfmrs
output_index=['trsf_pole_nb', 'prem_nb']
assert_all_PNs_found=True
df_mp_serial_number_col='mfr_devc_ser_nbr'
df_mp_prem_nb_col='prem_nb'
df_mp_install_time_col='inst_ts'
df_mp_removal_time_col='rmvl_ts'
df_mp_trsf_pole_nb_col='trsf_pole_nb'

#NEW
return_SNs_col='SNs' #Probably not neeeded
return_prem_nbs_col='prem_nbs'  #probably not needed in function arguments, but should be defined within function

In [59]:
necessary_mp_cols = [df_mp_serial_number_col, df_mp_prem_nb_col, df_mp_install_time_col, df_mp_removal_time_col]

In [60]:
trsf_pole_nbs

['38830119D30001',
 '39821038B10017',
 '39830544000129',
 '39830928D20281',
 '39831125A40001',
 '38820049000081',
 '38830021C20087',
 '39831149D10317',
 '39821038B10052',
 '39831124C30054']

In [61]:
# PROBLEM!!!!!
# MAKING SURE df_mp_hist and df_mp_curr have same form!!!!!

In [62]:
# SNs_and_PNs_for_xfrms, df_mp_curr = MeterPremise.get_SNs_for_xfmrs(
#     trsf_pole_nbs=trsf_pole_nbs, 
#     include_prem_nbs=True,
#     trsf_pole_nb_col=df_mp_trsf_pole_nb_col, 
#     serial_number_col=df_mp_serial_number_col, 
#     prem_nb_col=df_mp_prem_nb_col, 
#     return_SNs_col=return_SNs_col,
#     return_prem_nbs_col=return_prem_nbs_col, 
#     assert_all_trsf_pole_nbs_found=True,        
#     mp_df=df_mp_curr, 
#     return_mp_df_also=True
# )

# #-------------------------
# if df_mp_hist is None:
#     mp_hist = MeterPremise(
#         df_construct_type=DFConstructType.kRunSqlQuery, 
#         contstruct_df_args=None, 
#         init_df_in_constructor=True, 
#         build_sql_function = MeterPremise.build_sql_meter_premise, 
#         build_sql_function_kwargs = dict(
#             cols_of_interest=necessary_mp_cols,  
#             trsf_pole_nbs=trsf_pole_nbs, 
#             field_to_split='trsf_pole_nbs', 
#             batch_size=15000,
#             n_update=1000, 
#             table_name='meter_premise_hist'
#         )
#     )
#     df_mp_hist = mp_hist.df
# #-------------------------
# # At a bare minimum, df_mp_curr and df_mp_hist must both have the following columns:
# #   necessary_mp_cols = ['mfr_devc_ser_nbr', 'prem_nb', 'inst_ts', 'rmvl_ts']
# assert(all([x in df_mp_curr.columns for x in necessary_mp_cols]))
# assert(all([x in df_mp_hist.columns for x in necessary_mp_cols]))   
# #-------------------------
# PNs = Utilities_df.consolidate_column_of_lists(SNs_and_PNs_for_xfrms, return_prem_nbs_col)
# #-------------------------
# if return_consolidated:
#     output_index = None
#     output_groupby = None
# hist_SNs_for_xfmrs = MeterPremise.get_historic_SNs_for_PNs(
#     PNs=PNs, 
#     df_mp_curr=df_mp_curr, 
#     df_mp_hist=df_mp_hist, 
#     assume_one_xfmr_per_PN=True, 
#     output_index=output_index,
#     output_groupby=output_groupby,
#     assert_all_PNs_found=True, 
#     df_mp_serial_number_col=df_mp_serial_number_col, 
#     df_mp_prem_nb_col=df_mp_prem_nb_col, 
#     df_mp_install_time_col=df_mp_install_time_col, 
#     df_mp_removal_time_col=df_mp_removal_time_col, 
#     df_mp_trsf_pole_nb_col=df_mp_trsf_pole_nb_col
# )
# #-------------------------
# if return_consolidated:
#     hist_SNs_for_xfmrs = Utilities_df.consolidate_df(
#         df=hist_SNs_for_xfmrs, 
#         groupby_col=df_mp_trsf_pole_nb_col, 
#         cols_shared_by_group=None, 
#         cols_to_collect_in_lists=[df_mp_prem_nb_col, df_mp_serial_number_col], 
#         include_groupby_col_in_output_cols=False, 
#         allow_duplicates_in_lists=False, 
#         recover_uniqueness_violators=True, 
#         rename_cols=None, 
#         verbose=False
#     )
# #return hist_SNs_for_xfmrs

In [81]:
SNs_and_PNs_for_xfrms.equals(sns_for_xfrms)

False

In [82]:
return_consolidated

True

In [83]:
start=time.time()
res = MeterPremise.get_historic_SNs_for_PNs(
    PNs=PNs, 
    df_mp_curr=df_mp_curr_all, 
    df_mp_hist=df_mp_hist_all, 
    assume_one_xfmr_per_PN=True, 
    output_index=['trsf_pole_nb', 'prem_nb'],
    output_groupby=['trsf_pole_nb', 'prem_nb'],
    assert_all_PNs_found=True
)
print(time.time()-start)

1.9761688709259033


In [84]:
start=time.time()
res2 = MeterPremise.get_historic_SNs_for_PNs(
    PNs=PNs, 
    df_mp_curr=df_mp_curr_all, 
    df_mp_hist=df_mp_hist_all, 
    assume_one_xfmr_per_PN=True, 
    output_index=None,
    output_groupby=None,
    assert_all_PNs_found=True
)
print(time.time()-start)

1.86590576171875


In [85]:
res2

,mfr_devc_ser_nbr,longitude,latitude,state_cd,prem_nb,srvc_pole_nb,trsf_pole_nb,latitude_nb,longitude_nb,circuit_nb,...,xfmr_nb,annual_kwh,annual_max_dmnd,mtr_stat_cd,mtr_stat_cd_desc,devc_stat_cd,devc_stat_cd_desc,curr_acct_cls_cd,inst_ts,rmvl_ts
0,300023952,-82.53942764,38.4058079,OH,078036129,38830119D30001,38830119D30001,38.4058079,-82.53942764,,...,,75993.000,0.000,C,CONNECTED,A,ACTIVE,R,2018-10-18 12:00:20,NaT
1,300023953,-82.28310156,38.59069353,OH,073574480,39821038B10018,39821038B10017,38.59069353,-82.28310156,,...,,3102.000,0.000,C,CONNECTED,A,ACTIVE,P,2018-10-17 12:00:20,NaT
2,300023954,-82.85232000,39.031350000,OH,076684265,39830544000129,39830544000129,39.031350000,-82.85232000,,...,,26242.000,0.000,C,CONNECTED,A,ACTIVE,R,2018-10-18 12:00:20,NaT
3,300023959,-82.84455574,38.69047534,OH,074663922,39830928D20281,39830928D20281,38.69047534,-82.84455574,,...,,47585.000,0.000,C,CONNECTED,A,ACTIVE,R,2018-10-22 12:00:20,NaT
4,300023960,-82.64657000,38.536100000,OH,078364867,39831125A40044,39831125A40001,38.536100000,-82.64657000,,...,,22257.000,0.000,C,CONNECTED,A,ACTIVE,R,2018-10-23 12:00:20,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3102393,661558941,NaN,NaN,NaN,075763812,NaN,41830832A30028,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-02-24 12:00:20,2018-09-05 12:00:00
3106081,58524595,NaN,NaN,NaN,070385598,NaN,38830120A20210,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1973-10-01 00:00:00,2012-11-10 12:00:00
3106459,46366192,NaN,NaN,NaN,077675073,NaN,41820981D10019,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1964-09-01 00:00:00,2013-01-22 12:00:00
3106794,218032424,NaN,NaN,NaN,075294669,NaN,38820073D10055,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011-08-22 12:00:20,2012-11-08 12:00:00


In [86]:
res

trsf_pole_nb    prem_nb  
38820001A30022  079466245                                    [300033691]
38820025000052  072725502                         [218648410, 428013584]
                075547786                          [532575298, 89331585]
                078647786               [25183172, 300024028, 532589113]
38820026A40226  075126788                                    [300024042]
                                                ...                     
41850744000159  074016390    [190018621, 300033182, 66003813, 882746844]
41850791000118  076387815                                    [300033213]
41850791000241  071564087                                    [300033147]
41850836000078  070761135     [300033184, 47105199, 661696656, 99404806]
41850838000099  075522135    [300033826, 655788215, 65676904, 661696605]
Name: mfr_devc_ser_nbr, Length: 2040, dtype: object

In [87]:
# start=time.time()
# res = MeterPremise.get_active_SNs_for_PNs_at_datetime_interval(
#     PNs=PNs, 
#     df_mp_curr=df_mp_curr_all, 
#     df_mp_hist=df_mp_hist_all, 
#     dt_0=date_0, 
#     dt_1=date_1, 
#     assume_one_xfmr_per_PN=True, 
#     output_index=['trsf_pole_nb', 'prem_nb'],
#     output_groupby=['trsf_pole_nb', 'prem_nb'], 
#     include_prems_wo_active_SNs_when_groupby=True
# )
# print(time.time()-start)

In [88]:
res_df = res.to_frame()
res_df

mfr_devc_ser_nbr
trsf_pole_nb   prem_nb                                               
38820001A30022 079466245                                  [300033691]
38820025000052 072725502                       [218648410, 428013584]
               075547786                        [532575298, 89331585]
               078647786             [25183172, 300024028, 532589113]
38820026A40226 075126788                                  [300024042]
...                                                               ...
41850744000159 074016390  [190018621, 300033182, 66003813, 882746844]
41850791000118 076387815                                  [300033213]
41850791000241 071564087                                  [300033147]
41850836000078 070761135   [300033184, 47105199, 661696656, 99404806]
41850838000099 075522135  [300033826, 655788215, 65676904, 661696605]

[2040 rows x 1 columns]

In [89]:
res_df=res_df.reset_index()
res_df

,trsf_pole_nb,prem_nb,mfr_devc_ser_nbr
0,38820001A30022,079466245,[300033691]
1,38820025000052,072725502,"[218648410, 428013584]"
2,38820025000052,075547786,"[532575298, 89331585]"
3,38820025000052,078647786,"[25183172, 300024028, 532589113]"
4,38820026A40226,075126788,[300024042]
...,...,...,...
2035,41850744000159,074016390,"[190018621, 300033182, 66003813, 882746844]"
2036,41850791000118,076387815,[300033213]
2037,41850791000241,071564087,[300033147]
2038,41850836000078,070761135,"[300033184, 47105199, 661696656, 99404806]"


In [90]:
res_df=res_df.explode('mfr_devc_ser_nbr')
res_df

,trsf_pole_nb,prem_nb,mfr_devc_ser_nbr
0,38820001A30022,079466245,300033691
1,38820025000052,072725502,218648410
1,38820025000052,072725502,428013584
2,38820025000052,075547786,532575298
2,38820025000052,075547786,89331585
...,...,...,...
2038,41850836000078,070761135,99404806
2039,41850838000099,075522135,300033826
2039,41850838000099,075522135,655788215
2039,41850838000099,075522135,65676904


In [ ]:
(res_df.groupby('trsf_pole_nb')[['trsf_pole_nb']].nunique()<=1)

In [91]:
res_df

,trsf_pole_nb,prem_nb,mfr_devc_ser_nbr
0,38820001A30022,079466245,300033691
1,38820025000052,072725502,218648410
1,38820025000052,072725502,428013584
2,38820025000052,075547786,532575298
2,38820025000052,075547786,89331585
...,...,...,...
2038,41850836000078,070761135,99404806
2039,41850838000099,075522135,300033826
2039,41850838000099,075522135,655788215
2039,41850838000099,075522135,65676904


In [92]:
res_df_final = Utilities_df.consolidate_df(
    df=res_df, 
    groupby_col='trsf_pole_nb', 
    cols_shared_by_group=None, 
    cols_to_collect_in_lists=['prem_nb', 'mfr_devc_ser_nbr'], 
    include_groupby_col_in_output_cols=False, 
    allow_duplicates_in_lists=False, 
    recover_uniqueness_violators=True, 
    rename_cols=None, 
    verbose=True
)

groups_violating_uniqueness = []


In [93]:
res_df_final

,prem_nb,mfr_devc_ser_nbr
trsf_pole_nb,,
38820001A30022,[079466245],[300033691]
38820025000052,"[078647786, 075547786, 072725502]","[532589113, 428013584, 218648410, 89331585, 53..."
38820026A40226,[075126788],[300024042]
38820026C20015,"[079600325, 078010325]","[83118192, 300024070, 12877118, 656424134, 534..."
38820026D10046,[076134723],[300024009]
...,...,...
41850744000159,[074016390],"[300033182, 190018621, 66003813, 882746844]"
41850791000118,[076387815],[300033213]
41850791000241,[071564087],[300033147]


In [ ]:
res_df_final = res_df_final.set_index('trsf_pole_nb')
res_df_final = res_df_final.sort_index()
res_df_final['prem_nb'] = res_df_final['prem_nb'].apply(sorted)
res_df_final['mfr_devc_ser_nbr'] = res_df_final['mfr_devc_ser_nbr'].apply(sorted)

In [ ]:
sns_for_xfrms = sns_for_xfrms.sort_index()
sns_for_xfrms['SNs'] = sns_for_xfrms['SNs'].apply(sorted)
sns_for_xfrms['prem_nbs'] = sns_for_xfrms['prem_nbs'].apply(sorted)
sns_for_xfrms = sns_for_xfrms.rename(columns={'SNs':'mfr_devc_ser_nbr', 'prem_nbs':'prem_nb'})
sns_for_xfrms = sns_for_xfrms[res_df_final.columns]

In [ ]:
# res_df_final.equals(sns_for_xfrms)

In [ ]:
res_df_final.explode('prem_nb').explode('mfr_devc_ser_nbr')

In [ ]:
res_df_final

In [ ]:
res2

In [ ]:
output_index=['trsf_pole_nb', 'prem_nb']
assert_all_PNs_found=True
df_mp_serial_number_col='mfr_devc_ser_nbr'
df_mp_prem_nb_col='prem_nb'
df_mp_install_time_col='inst_ts'
df_mp_removal_time_col='rmvl_ts'
df_mp_trsf_pole_nb_col='trsf_pole_nb'

In [ ]:
res_df_final2 = Utilities_df.consolidate_df(
    df=res2, 
    groupby_col=df_mp_trsf_pole_nb_col, 
    cols_shared_by_group=None, 
    cols_to_collect_in_lists=[df_mp_prem_nb_col, df_mp_serial_number_col], 
    include_groupby_col_in_output_cols=False, 
    allow_duplicates_in_lists=False, 
    recover_uniqueness_violators=True, 
    rename_cols=None, 
    verbose=True
)

In [ ]:
res_df_final2

In [ ]:
res_df_final2.equals(res_df_final)

In [ ]:
res_df_final = res_df_final.sort_index()
res_df_final['prem_nb'] = res_df_final['prem_nb'].apply(sorted)
res_df_final['mfr_devc_ser_nbr'] = res_df_final['mfr_devc_ser_nbr'].apply(sorted)

In [ ]:
res_df_final2 = res_df_final2.sort_index()
res_df_final2['prem_nb'] = res_df_final2['prem_nb'].apply(sorted)
res_df_final2['mfr_devc_ser_nbr'] = res_df_final2['mfr_devc_ser_nbr'].apply(sorted)

In [ ]:
res_df_final2.equals(res_df_final)

In [ ]:
hist_PNs_and_SNs_for_xfmrs = get_historic_PNs_and_SNs_for_xfmrs(
    trsf_pole_nbs, 
    df_mp_curr, 
    df_mp_hist
)

hist_PNs_and_SNs_for_xfmrs = hist_PNs_and_SNs_for_xfmrs.sort_index()
hist_PNs_and_SNs_for_xfmrs['PNs'] = hist_PNs_and_SNs_for_xfmrs['PNs'].apply(sorted)
hist_PNs_and_SNs_for_xfmrs['SNs'] = hist_PNs_and_SNs_for_xfmrs['SNs'].apply(sorted)
hist_PNs_and_SNs_for_xfmrs = hist_PNs_and_SNs_for_xfmrs.rename(columns={'PNs':'prem_nb', 'SNs':'mfr_devc_ser_nbr'})

In [ ]:
hist_PNs_and_SNs_for_xfmrs.equals(res_df_final2)

In [ ]:
hist_PNs_and_SNs_for_xfmrs

In [2]:
import os

def countlines(start, lines=0, header=True, begin_start=None):
    if header:
        print('{:>10} |{:>10} | {:<20}'.format('ADDED', 'TOTAL', 'FILE'))
        print('{:->11}|{:->11}|{:->20}'.format('', '', ''))

    for thing in os.listdir(start):
        thing = os.path.join(start, thing)
        if os.path.isfile(thing):
            if thing.endswith('.py'):
                with open(thing, 'r') as f:
                    newlines = f.readlines()
                    newlines = len(newlines)
                    lines += newlines

                    if begin_start is not None:
                        reldir_of_thing = '.' + thing.replace(begin_start, '')
                    else:
                        reldir_of_thing = '.' + thing.replace(start, '')

                    print('{:>10} |{:>10} | {:<20}'.format(
                            newlines, lines, reldir_of_thing))


    for thing in os.listdir(start):
        thing = os.path.join(start, thing)
        if os.path.isdir(thing):
            lines = countlines(thing, lines, header=False, begin_start=start)

    return lines

In [3]:
countlines(r'C:\Users\s346557\Documents\Analysis')

     ADDED |     TOTAL | FILE                
-----------|-----------|--------------------
      1457 |      1457 | .\AMIEDE_DEV.py     
      3176 |      4633 | .\AMIEndEvents.py   
       348 |      4981 | .\AMIEndEvents_SQL.py
      1051 |      6032 | .\AMINonVee.py      
      1237 |      7269 | .\AMINonVee_SQL.py  
       107 |      7376 | .\AMIUsgInst.py     
       244 |      7620 | .\AMIUsgInst_SQL.py 
      1188 |      8808 | .\AMI_SQL.py        
       811 |      9619 | .\CommonLearningMethods.py
      2236 |     11855 | .\DOVSOutages.py    
       907 |     12762 | .\DOVSOutages_SQL.py
       327 |     13089 | .\EEMSP.py          
       864 |     13953 | .\GenAn.py          
      1863 |     15816 | .\MECPOAn.py        
      1587 |     17403 | .\MECPOCollection.py
      2665 |     20068 | .\MECPODf.py        
      2047 |     22115 | .\MeterPremise.py   
       174 |     22289 | .\Utilities_config.py
        36 |     22325 | .\NISTResultHandler\CTNConsistencyResults.py
   

38918

In [2]:
print("WITH USG_gnrl AS (\n\tSELECT\n\t\tun_rin.issuertracking_id,\n\t\tun_rin.serialnumber,\n\t\tun_rin.enddeviceeventtypeid,\n\t\tun_rin.valuesinterval,\n\t\tun_rin.aep_premise_nb,\n\t\tun_rin.reason,\n\t\tun_rin.event_type,\n\t\tun_rin.aep_opco,\n\t\tun_rin.aep_event_dt\n\tFROM meter_events.end_device_event un_rin\n\tWHERE un_rin.aep_opco = 'oh'\n\tAND   (\n\t\tun_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\t)\n), \nUSG_0 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580837' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'1863938736669' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:19:00' AND TIMESTAMP '2022-01-31 03:24:00'\n\tAND   un_rin.aep_premise_nb IN ('101407483','101717483','101904210','103414210','103696483','103907483','104726048','105107483','105417483','105604210','106017483','107114210','107607483','107893210','108024210','109304210','109614210','109896483')\n), \nUSG_1 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580852' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40821026000237' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 04:21:00'\n\tAND   un_rin.aep_premise_nb = '075163797'\n), \nUSG_2 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580864' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'41840918000092' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 03:21:00' AND TIMESTAMP '2022-01-31 04:45:00'\n\tAND   un_rin.aep_premise_nb = '073045377'\n), \nUSG_3 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580898' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40820599C40015' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 07:26:00' AND TIMESTAMP '2022-01-31 07:46:00'\n\tAND   un_rin.aep_premise_nb = '075901475'\n), \nUSG_4 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580900' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40830909D10002' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 01:41:00' AND TIMESTAMP '2022-01-31 06:57:00'\n\tAND   un_rin.aep_premise_nb IN ('070488870','071088870','072678870','073588870','074188870','075778870','076688870','077288870','078878870','079478870','079788870')\n), \nUSG_5 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580900' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40830909D10041' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 01:41:00' AND TIMESTAMP '2022-01-31 06:57:00'\n\tAND   un_rin.aep_premise_nb IN ('071768870','072368870','073958870','074868870','075468870','076068870','077968870','078568870','079168870')\n), \nUSG_6 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580900' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40830909D30077' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 01:41:00' AND TIMESTAMP '2022-01-31 06:57:00'\n\tAND   un_rin.aep_premise_nb IN ('071242830','071971970','072571970','072832830','073171970','074081970','074342830','075671970','075932830','076271970','078612970','078771970','079371970')\n), \nUSG_7 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580900' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40830909D30080' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 01:41:00' AND TIMESTAMP '2022-01-31 06:57:00'\n\tAND   un_rin.aep_premise_nb IN ('070071970','070642830','071661970','072152830','073742830','074761970','075252830','076842830','077442830','077861970','079942830')\n), \nUSG_8 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580900' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40830909D30081' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 01:41:00' AND TIMESTAMP '2022-01-31 06:57:00'\n\tAND   un_rin.aep_premise_nb IN ('075361970','078352830','078461970')\n), \nUSG_9 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580900' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40830909D30084' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 01:41:00' AND TIMESTAMP '2022-01-31 06:57:00'\n\tAND   un_rin.aep_premise_nb IN ('070491830','073591830','076691830','077291830','079791830')\n), \nUSG_10 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580900' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40830909D30090' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 01:41:00' AND TIMESTAMP '2022-01-31 06:57:00'\n\tAND   un_rin.aep_premise_nb IN ('070272830','071552830','073372830','073898870','074652830','074962830','077072830')\n), \nUSG_11 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580900' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40830909D30095' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 01:41:00' AND TIMESTAMP '2022-01-31 06:57:00'\n\tAND   un_rin.aep_premise_nb IN ('070178870','071862830','072462830','075562830','076472830','078662830','079262830')\n), \nUSG_12 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580900' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40830909D30099' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 01:41:00' AND TIMESTAMP '2022-01-31 06:57:00'\n\tAND   un_rin.aep_premise_nb IN ('071302830','072991830','074402830','075002830','076288611','077502830')\n), \nUSG_13 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580900' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40830909D30115' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 01:41:00' AND TIMESTAMP '2022-01-31 06:57:00'\n\tAND   un_rin.aep_premise_nb = '070019870'\n), \nUSG_14 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580900' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40830909D30126' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 01:41:00' AND TIMESTAMP '2022-01-31 06:57:00'\n\tAND   un_rin.aep_premise_nb IN ('070702830','072212830','073802830','075312830','076902830','078412830','079012830')\n), \nUSG_15 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580900' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40830909D30130' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 01:41:00' AND TIMESTAMP '2022-01-31 06:57:00'\n\tAND   un_rin.aep_premise_nb IN ('070241830','071173535','071612830','071831830','074712830','077041830','077812830','079020383','079316528')\n), \nUSG_16 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580900' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40830909D30135' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 01:41:00' AND TIMESTAMP '2022-01-31 06:57:00'\n\tAND   un_rin.aep_premise_nb IN ('070798870','070952830','077752830')\n), \nUSG_17 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580900' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40830909D30136' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 01:41:00' AND TIMESTAMP '2022-01-31 06:57:00'\n\tAND   un_rin.aep_premise_nb IN ('073062830','074498870','076162830','077598870')\n), \nUSG_18 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580900' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40830909D30137' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 01:41:00' AND TIMESTAMP '2022-01-31 06:57:00'\n\tAND   un_rin.aep_premise_nb IN ('071398870','072988870','073278870','076378870','078198870')\n), \nUSG_19 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580900' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40830909D30168' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 01:41:00' AND TIMESTAMP '2022-01-31 06:57:00'\n\tAND   un_rin.aep_premise_nb IN ('071091830','072772830','074191830','075872830','078881830','079572830')\n), \nUSG_20 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580900' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40830909D30170' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 01:41:00' AND TIMESTAMP '2022-01-31 06:57:00'\n\tAND   un_rin.aep_premise_nb IN ('071182830','072681830','075781830','078972830')\n), \nUSG_21 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580900' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40830909D30171' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 01:41:00' AND TIMESTAMP '2022-01-31 06:57:00'\n\tAND   un_rin.aep_premise_nb IN ('074282830','076381830','077382830','079481830')\n), \nUSG_22 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580900' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40830909D30174' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 01:41:00' AND TIMESTAMP '2022-01-31 06:57:00'\n\tAND   un_rin.aep_premise_nb IN ('070582830','071007285','073281830','073682830')\n), \nUSG_23 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580900' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40830909D30175' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 01:41:00' AND TIMESTAMP '2022-01-31 06:57:00'\n\tAND   un_rin.aep_premise_nb IN ('074871830','077971830','078571830')\n), \nUSG_24 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580901' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40820929000032' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 06:18:00' AND TIMESTAMP '2022-01-31 07:55:00'\n\tAND   un_rin.aep_premise_nb = '076397288'\n), \nUSG_25 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580901' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40820929000043' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 06:18:00' AND TIMESTAMP '2022-01-31 07:55:00'\n\tAND   un_rin.aep_premise_nb = '073376532'\n), \nUSG_26 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40810577000044' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb = '071817295'\n), \nUSG_27 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40810577000050' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb IN ('070133501','070227295')\n), \nUSG_28 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40810577000053' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb = '077027295'\n), \nUSG_29 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40810577000054' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb = '074917295'\n), \nUSG_30 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40810577000127' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb = '075827295'\n), \nUSG_31 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40810577000128' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb = '073327295'\n), \nUSG_32 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40810577000191' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb = '075351940'\n), \nUSG_33 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40810601000145' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb = '075608295'\n), \nUSG_34 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40810601000148' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb = '078708295'\n), \nUSG_35 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40810601000171' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb IN ('071908295','074018295')\n), \nUSG_36 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40810601000174' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb = '073108295'\n), \nUSG_37 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40810601000175' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb IN ('076208295','079308295')\n), \nUSG_38 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40810601000176' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb = '079366216'\n), \nUSG_39 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40810601000236' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb = '078898295'\n), \nUSG_40 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40810601000260' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb = '078247295'\n), \nUSG_41 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40810601000263' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb = '071447295'\n), \nUSG_42 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40810601000273' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb = '072508295'\n), \nUSG_43 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40810601000284' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb IN ('074547295','077647295')\n), \nUSG_44 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40810601000290' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb IN ('074246395','077346395')\n), \nUSG_45 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40810601000291' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb IN ('070546395','072015062','075008860')\n), \nUSG_46 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40810601000293' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb IN ('071146395','076444547')\n), \nUSG_47 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40810601000311' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb IN ('070716684','071347811','073160308')\n), \nUSG_48 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40810601000313' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb = '074386311'\n), \nUSG_49 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40810601000314' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb = '073833337'\n), \nUSG_50 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40810601000316' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb IN ('076994133','079114958')\n), \nUSG_51 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40810601000335' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb = '075147295'\n), \nUSG_52 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40810601000381' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb = '071290389'\n), \nUSG_53 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40810625000073' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb = '071979362'\n), \nUSG_54 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40810625000074' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb IN ('073429110','079618295')\n), \nUSG_55 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40810625000253' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb = '076421305'\n), \nUSG_56 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40810626000032' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb = '076828295'\n), \nUSG_57 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40810626000039' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb = '074625207'\n), \nUSG_58 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40810626000040' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb = '078839627'\n), \nUSG_59 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40810626000041' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb = '079928295'\n), \nUSG_60 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40810626000049' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb = '078028295'\n), \nUSG_61 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40810626000090' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb = '072766507'\n), \nUSG_62 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40810626000098' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb = '078815946'\n), \nUSG_63 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40810626000101' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb = '079708918'\n), \nUSG_64 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40810626000106' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb = '079245865'\n), \nUSG_65 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40820576000292' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb IN ('070355083','076565024')\n), \nUSG_66 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40820576000293' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb = '071795591'\n), \nUSG_67 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40820576000295' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb = '078075024'\n), \nUSG_68 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40820576000298' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb = '070644357'\n), \nUSG_69 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40820576000313' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb = '078502859'\n), \nUSG_70 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40820576000326' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb = '078576378'\n), \nUSG_71 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40820576000334' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb = '076427355'\n), \nUSG_72 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40820576000456' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb = '075965024'\n), \nUSG_73 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40820576000467' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb = '071275024'\n), \nUSG_74 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40820576000468' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb = '075930313'\n), \nUSG_75 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40820600000004' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb = '077475024'\n), \nUSG_76 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40820600C30018' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb IN ('071757355','074857355','075457355','077957355','078557355')\n), \nUSG_77 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40820600C30020' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb IN ('070167355','072667355','073267355','075767355','076367355','079467355')\n), \nUSG_78 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40820600C30024' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb IN ('070537355','071447355','072047355','073637355','076737355','077337355')\n), \nUSG_79 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40820600C30026' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb IN ('071137355','072727355','074237355','075827355')\n), \nUSG_80 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40820600C40009' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb IN ('072357355','078867355','079157355')\n), \nUSG_81 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40820600C40012' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb IN ('070847355','071485226','073947355','074547355','078247355')\n), \nUSG_82 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40820600C40020' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb = '074177355'\n), \nUSG_83 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40820600C40021' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb IN ('070477355','073293683','077277355')\n), \nUSG_84 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40820600C40022' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb = '073577355'\n), \nUSG_85 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40820600C40023' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb IN ('071390789','072977355','078187355')\n), \nUSG_86 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40820600C40030' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb = '077587355'\n), \nUSG_87 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40820600C40031' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb = '070787355'\n), \nUSG_88 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40820600C40039' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb = '076987355'\n), \nUSG_89 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40820600C40041' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb = '071077355'\n), \nUSG_90 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40820600C40051' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb = '074487355'\n), \nUSG_91 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40820600C40052' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb = '071387355'\n), \nUSG_92 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40820600C40061' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb = '073887355'\n), \nUSG_93 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40820600C40064' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb IN ('078066561','079097355')\n), \nUSG_94 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580903' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40820600C40234' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:29:00' AND TIMESTAMP '2022-01-31 06:54:00'\n\tAND   un_rin.aep_premise_nb = '078714141'\n), \nUSG_95 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580960' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'38820051B30045' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 06:55:00' AND TIMESTAMP '2022-01-31 09:30:00'\n\tAND   un_rin.aep_premise_nb IN ('072406516','076099092')\n), \nUSG_96 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580981' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40820846000007' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 08:30:00' AND TIMESTAMP '2022-01-31 10:05:00'\n\tAND   un_rin.aep_premise_nb = '072023305'\n), \nUSG_97 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580981' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40820846000020' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 08:30:00' AND TIMESTAMP '2022-01-31 10:05:00'\n\tAND   un_rin.aep_premise_nb IN ('074801355','077901355')\n), \nUSG_98 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580981' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40820846000028' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 08:30:00' AND TIMESTAMP '2022-01-31 10:05:00'\n\tAND   un_rin.aep_premise_nb = '070111355'\n), \nUSG_99 AS (\n\tSELECT\n\t\tun_rin.*,\n\t\t'12580981' AS OUTG_REC_NB_GPD_FOR_SQL,\n\t\t'40820846000031' AS trsf_pole_nb_GPD_FOR_SQL\n\tFROM USG_gnrl un_rin\n\tWHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'\n\tAND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 08:30:00' AND TIMESTAMP '2022-01-31 10:05:00'\n\tAND   un_rin.aep_premise_nb = '076743737'\n)\n\nSELECT * FROM USG_0\nUNION\nSELECT * FROM USG_1\nUNION\nSELECT * FROM USG_2\nUNION\nSELECT * FROM USG_3\nUNION\nSELECT * FROM USG_4\nUNION\nSELECT * FROM USG_5\nUNION\nSELECT * FROM USG_6\nUNION\nSELECT * FROM USG_7\nUNION\nSELECT * FROM USG_8\nUNION\nSELECT * FROM USG_9\nUNION\nSELECT * FROM USG_10\nUNION\nSELECT * FROM USG_11\nUNION\nSELECT * FROM USG_12\nUNION\nSELECT * FROM USG_13\nUNION\nSELECT * FROM USG_14\nUNION\nSELECT * FROM USG_15\nUNION\nSELECT * FROM USG_16\nUNION\nSELECT * FROM USG_17\nUNION\nSELECT * FROM USG_18\nUNION\nSELECT * FROM USG_19\nUNION\nSELECT * FROM USG_20\nUNION\nSELECT * FROM USG_21\nUNION\nSELECT * FROM USG_22\nUNION\nSELECT * FROM USG_23\nUNION\nSELECT * FROM USG_24\nUNION\nSELECT * FROM USG_25\nUNION\nSELECT * FROM USG_26\nUNION\nSELECT * FROM USG_27\nUNION\nSELECT * FROM USG_28\nUNION\nSELECT * FROM USG_29\nUNION\nSELECT * FROM USG_30\nUNION\nSELECT * FROM USG_31\nUNION\nSELECT * FROM USG_32\nUNION\nSELECT * FROM USG_33\nUNION\nSELECT * FROM USG_34\nUNION\nSELECT * FROM USG_35\nUNION\nSELECT * FROM USG_36\nUNION\nSELECT * FROM USG_37\nUNION\nSELECT * FROM USG_38\nUNION\nSELECT * FROM USG_39\nUNION\nSELECT * FROM USG_40\nUNION\nSELECT * FROM USG_41\nUNION\nSELECT * FROM USG_42\nUNION\nSELECT * FROM USG_43\nUNION\nSELECT * FROM USG_44\nUNION\nSELECT * FROM USG_45\nUNION\nSELECT * FROM USG_46\nUNION\nSELECT * FROM USG_47\nUNION\nSELECT * FROM USG_48\nUNION\nSELECT * FROM USG_49\nUNION\nSELECT * FROM USG_50\nUNION\nSELECT * FROM USG_51\nUNION\nSELECT * FROM USG_52\nUNION\nSELECT * FROM USG_53\nUNION\nSELECT * FROM USG_54\nUNION\nSELECT * FROM USG_55\nUNION\nSELECT * FROM USG_56\nUNION\nSELECT * FROM USG_57\nUNION\nSELECT * FROM USG_58\nUNION\nSELECT * FROM USG_59\nUNION\nSELECT * FROM USG_60\nUNION\nSELECT * FROM USG_61\nUNION\nSELECT * FROM USG_62\nUNION\nSELECT * FROM USG_63\nUNION\nSELECT * FROM USG_64\nUNION\nSELECT * FROM USG_65\nUNION\nSELECT * FROM USG_66\nUNION\nSELECT * FROM USG_67\nUNION\nSELECT * FROM USG_68\nUNION\nSELECT * FROM USG_69\nUNION\nSELECT * FROM USG_70\nUNION\nSELECT * FROM USG_71\nUNION\nSELECT * FROM USG_72\nUNION\nSELECT * FROM USG_73\nUNION\nSELECT * FROM USG_74\nUNION\nSELECT * FROM USG_75\nUNION\nSELECT * FROM USG_76\nUNION\nSELECT * FROM USG_77\nUNION\nSELECT * FROM USG_78\nUNION\nSELECT * FROM USG_79\nUNION\nSELECT * FROM USG_80\nUNION\nSELECT * FROM USG_81\nUNION\nSELECT * FROM USG_82\nUNION\nSELECT * FROM USG_83\nUNION\nSELECT * FROM USG_84\nUNION\nSELECT * FROM USG_85\nUNION\nSELECT * FROM USG_86\nUNION\nSELECT * FROM USG_87\nUNION\nSELECT * FROM USG_88\nUNION\nSELECT * FROM USG_89\nUNION\nSELECT * FROM USG_90\nUNION\nSELECT * FROM USG_91\nUNION\nSELECT * FROM USG_92\nUNION\nSELECT * FROM USG_93\nUNION\nSELECT * FROM USG_94\nUNION\nSELECT * FROM USG_95\nUNION\nSELECT * FROM USG_96\nUNION\nSELECT * FROM USG_97\nUNION\nSELECT * FROM USG_98\nUNION\nSELECT * FROM USG_99",)

WITH USG_gnrl AS (
	SELECT
		un_rin.issuertracking_id,
		un_rin.serialnumber,
		un_rin.enddeviceeventtypeid,
		un_rin.valuesinterval,
		un_rin.aep_premise_nb,
		un_rin.reason,
		un_rin.event_type,
		un_rin.aep_opco,
		un_rin.aep_event_dt
	FROM meter_events.end_device_event un_rin
	WHERE un_rin.aep_opco = 'oh'
	AND   (
		un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'
	)
), 
USG_0 AS (
	SELECT
		un_rin.*,
		'12580837' AS OUTG_REC_NB_GPD_FOR_SQL,
		'1863938736669' AS trsf_pole_nb_GPD_FOR_SQL
	FROM USG_gnrl un_rin
	WHERE un_rin.aep_event_dt BETWEEN '2021-12-02' AND '2022-01-31'
	AND   CAST(regexp_replace(un_rin.valuesinterval, '([0-9]{4}-[0-9]{2}-[0-9]{2})T([0-9]{2}:[0-9]{2}:[0-9]{2}).*', '$1 $2') AS TIMESTAMP) BETWEEN TIMESTAMP '2021-12-02 02:19:00' AND TIMESTAMP '2022-01-31 03:24:00'
	AND   un_rin.aep_premise_nb IN ('101407483','101717483','101904210','103414210','103696483','103907483','104726048','105107483','105417483','105604210','106017483','107114210','107607483','107893